In [ ]:
pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e4c36f1fcd769c5a49686e1cf88964698ca4deb9bb79866c58e0cc2ca5af4791
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


Baseline GPT2


In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer, AutoModelForSequenceClassification
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Baseline Model Functions ======================
def load_baseline_gpt2(device='cuda'):
    """Load vanilla GPT-2 model without custom tokenizer"""
    print("Loading vanilla GPT-2 model...")

    # Load standard GPT-2 tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.to(device)

    print(f"Vanilla GPT-2 loaded with vocab size: {len(tokenizer)}")

    return model, tokenizer

def load_emotion_classifier(device='cuda'):
    """Load RoBERTa emotion classifier"""
    print("Loading emotion classifier...")
    emotion_tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
    emotion_model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")
    emotion_model.to(device)
    emotion_model.eval()

    return emotion_model, emotion_tokenizer

def extract_user_input_for_baseline(input_text, use_full_input=False):
    """
    Extract input for baseline GPT-2

    Args:
        input_text: Full structured input
        use_full_input: If True, use full input (GPT-2 will see unknown tokens)
                       If False, extract just user text (default)
    """
    if use_full_input:
        # Return full input
        return input_text

    # Extract just the user text (default behavior for fair comparison)
    user_match = re.search(r'<user>(.*?)(?:<user_emotion>|<therapist>)', input_text)
    if user_match:
        user_text = user_match.group(1).strip()
        return user_text

    # Fallback: try to extract any text after <user>
    user_match = re.search(r'<user>(.*)', input_text)
    if user_match:
        return user_match.group(1).strip()

    # Last fallback: return the input as is
    return input_text

def predict_emotion_roberta(text, emotion_model, emotion_tokenizer, device='cuda'):
    """Predict emotion using RoBERTa model, constrained to therapy emotions"""
    if not text.strip():
        return "neutral", 0.0

    # Define therapy emotions we care about
    therapy_emotions = ['anger', 'joy', 'neutral', 'sadness', 'depression', 'disgust', 'fear']

    # Tokenize and predict
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = emotion_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Get all emotion labels from RoBERTa model
    emotion_labels = emotion_model.config.id2label

    emotion_mapping = {
        # Direct matches
        'anger': 'anger',
        'joy': 'joy',
        'sadness': 'sadness',
        'fear': 'fear',
        'disgust': 'disgust',
        'neutral': 'neutral',

        # Joy-related emotions
        'amusement': 'joy',
        'excitement': 'joy',
        'gratitude': 'joy',
        'love': 'joy',
        'optimism': 'joy',
        'pride': 'joy',
        'relief': 'joy',
        'admiration': 'joy',

        # Sadness-related emotions (potential depression indicators)
        'disappointment': 'sadness',
        'embarrassment': 'sadness',
        'grief': 'sadness',
        'remorse': 'sadness',

        # Anger-related emotions
        'annoyance': 'anger',
        'disapproval': 'anger',

        # Fear-related emotions
        'nervousness': 'fear',

        # Neutral-related emotions
        'approval': 'neutral',
        'caring': 'neutral',
        'confusion': 'neutral',
        'curiosity': 'neutral',
        'desire': 'neutral',
        'realization': 'neutral',
        'surprise': 'neutral'
    }

    # Aggregate scores for therapy emotions
    therapy_scores = {emotion: 0.0 for emotion in therapy_emotions}

    for class_id, score in enumerate(predictions[0]):
        go_emotion = emotion_labels[class_id]
        therapy_emotion = emotion_mapping.get(go_emotion, 'neutral')
        therapy_scores[therapy_emotion] += score.item()

    # Find the therapy emotion with highest score
    best_emotion = max(therapy_scores, key=therapy_scores.get)
    best_score = therapy_scores[best_emotion]

    # Enhanced depression detection based on strong indicators
    if best_emotion == 'sadness':
        depression_indicators = ['grief', 'remorse', 'disappointment', 'embarrassment']
        depression_score = sum(predictions[0][class_id].item()
                              for class_id, label in emotion_labels.items()
                              if label in depression_indicators)

        if depression_score > 0.2 or (emotion_labels.get(predictions[0].argmax().item()) in ['grief', 'remorse']):
            best_emotion = 'depression'
            best_score = depression_score

    return best_emotion, best_score

# ====================== Reference Data Processing ======================
class TherapyEvaluationDataset:
    """Dataset class for evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        self.evaluation_data = []
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation"""
        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input prompt
                    input_parts = []
                    if problem_type:
                        input_parts.append(f"<problem>{problem_type}")

                    input_parts.append(f"<user>{combined_user_text}")
                    if last_user_emotion:
                        input_parts.append(f"<user_emotion>{last_user_emotion}")

                    input_parts.append("<therapist>")
                    input_text = " ".join(input_parts)

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'reference_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Model Evaluation ======================
def evaluate_baseline(evaluation_dataset, device='cuda', top_p=0.8, max_new_tokens=128, use_full_input=False):
    """
    Evaluate vanilla GPT-2 model on therapy dataset with emotion detection

    Args:
        evaluation_dataset: TherapyEvaluationDataset instance
        device: Device to run evaluation on
        top_p: Top-p sampling parameter
        max_new_tokens: Maximum tokens to generate
        use_full_input: If True, feed full structured input to GPT-2
                       If False, extract just user text (default for fair comparison)

    Returns:
        Dictionary containing evaluation results
    """
    # Load baseline model and emotion classifier
    model, tokenizer = load_baseline_gpt2(device)
    emotion_model, emotion_tokenizer = load_emotion_classifier(device)

    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
        'sample_outputs': []
    }

    emotion_confusion = defaultdict(lambda: defaultdict(int))

    model.eval()

    if use_full_input:
        print("Evaluating vanilla GPT-2 with FULL structured input (will see unknown tokens)...")
    else:
        print("Evaluating vanilla GPT-2 with extracted user text only...")
    print(f"Evaluating model on {len(evaluation_data)} samples...")

    # Create progress bar
    progress_bar = tqdm(evaluation_data, desc="Evaluating Baseline",
                       bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')

    for i, sample in enumerate(progress_bar):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Extract user input for baseline
        user_input = extract_user_input_for_baseline(input_text, use_full_input)

        # Tokenize input for vanilla GPT-2
        input_ids = tokenizer.encode(user_input, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_p=top_p,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        predicted_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

        # Predict emotion using RoBERTa
        predicted_emotion, emotion_confidence = predict_emotion_roberta(
            predicted_text, emotion_model, emotion_tokenizer, device
        )

        # Compute text generation metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower())
        has_emotion = True

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

        # Update confusion matrix
        emotion_confusion[reference_emotion.lower()][predicted_emotion.lower()] += 1

        # Store sample outputs for inspection
        if i < 10:  # Store first 10 samples
            results['sample_outputs'].append({
                'input': input_text,
                'user_input_extracted': user_input,
                'reference_text': reference_text,
                'predicted_text': predicted_text,
                'reference_emotion': reference_emotion,
                'predicted_emotion': predicted_emotion,
                'emotion_confidence': emotion_confidence,
                'full_response': predicted_text,
                'bleu': bleu,
                'rouge1': rouge['rouge1'],
                'meteor': meteor
            })

        if (i + 1) % 20 == 0 or i < 5:
            sample_info = f"Sample {i+1}: BLEU={bleu:.3f}, Emotion={'✓' if emotion_correct else '✗'}"
            progress_bar.set_description(f"Evaluating Baseline - {sample_info}")

            if (i + 1) % 50 == 0 or i < 3:
                print(f"\n" + "="*80)
                print(f"BASELINE SAMPLE {i+1}/{len(evaluation_data)}")
                print("="*80)
                print(f"Original Input: {input_text[:120]}...")
                print(f"Processed Input: {user_input[:100]}...")
                if use_full_input:
                    print(f"Note: GPT-2 sees unknown tokens like <problem>, <user>, etc.")
                print(f"Reference: {reference_text[:100]}...")
                print(f"Generated: {predicted_text[:100]}...")
                print(f"Ref Emotion: {reference_emotion} | Pred Emotion: {predicted_emotion} (conf: {emotion_confidence:.3f}) | Match: {'✓' if emotion_correct else '✗'}")
                print(f"BLEU: {bleu:.4f} | ROUGE-1: {rouge['rouge1']:.4f} | METEOR: {meteor:.4f}")
                print("="*80)
        else:
            if i > 0:
                avg_bleu = np.mean(results['bleu_scores'])
                avg_emotion_acc = np.mean(results['emotion_accuracy'])
                progress_bar.set_description(f"Evaluating Baseline - Avg BLEU: {avg_bleu:.3f}, Emotion Acc: {avg_emotion_acc:.3f}")

    # Close progress bar
    progress_bar.close()

    # Compute summary statistics
    results['summary'] = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data)
    }

    results['emotion_confusion_matrix'] = dict(emotion_confusion)
    results['detected_format'] = 'roberta_emotion_detection'

    # Print final progress summary
    print(f"\n{'='*60}")
    print(f"BASELINE EVALUATION COMPLETED!")
    print(f"{'='*60}")
    print(f"Processed {len(evaluation_data)} samples")
    print(f"Average BLEU: {results['summary']['avg_bleu']:.4f}")
    print(f"Average ROUGE-1: {results['summary']['avg_rouge1']:.4f}")
    print(f"Emotion Accuracy: {results['summary']['emotion_accuracy']:.4f}")
    print(f"Emotion Tag Coverage: {results['summary']['emotion_tag_coverage']:.4f}")
    print(f"{'='*60}")

    results['model_type'] = 'baseline_gpt2_roberta_emotion'
    if use_full_input:
        results['model_type'] += '_full_input'
    else:
        results['model_type'] += '_user_only'

    results['generation_params'] = {
        'top_p': top_p,
        'max_new_tokens': max_new_tokens,
        'use_full_input': use_full_input
    }

    return results

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    summary = results['summary']

    print("\n" + "="*60)
    print("BASELINE GPT-2 EVALUATION RESULTS")
    print("="*60)
    print(f"Model Type: {results.get('model_type', 'Unknown').upper()}")
    print(f"Output Format: {results.get('detected_format', 'Unknown').upper()}")

    # Show generation parameters
    gen_params = results.get('generation_params', {})
    if gen_params:
        print(f"Generation Params: top_p={gen_params.get('top_p', 'N/A')}, "
              f"max_tokens={gen_params.get('max_new_tokens', 'N/A')}, "
              f"full_input={gen_params.get('use_full_input', 'N/A')}")

    print(f"\nText Generation Metrics:")
    print(f"  BLEU Score:     {summary['avg_bleu']:.4f}")
    print(f"  ROUGE-1:        {summary['avg_rouge1']:.4f}")
    print(f"  ROUGE-2:        {summary['avg_rouge2']:.4f}")
    print(f"  ROUGE-L:        {summary['avg_rougeL']:.4f}")
    print(f"  METEOR:         {summary['avg_meteor']:.4f}")

    print(f"\nEmotion Prediction:")
    print(f"  Emotion Accuracy:    {summary['emotion_accuracy']:.4f}")
    print(f"  Emotion Tag Coverage: {summary['emotion_tag_coverage']:.4f}")

    print(f"\nDataset Info:")
    print(f"  Total Samples:  {summary['total_samples']}")

    print(f"\nSample Outputs:")
    for i, sample in enumerate(results['sample_outputs'][:3], 1):
        print(f"\n  Sample {i}:")
        print(f"    Original Input: {sample['input'][:100]}...")
        print(f"    Processed Input: {sample['user_input_extracted'][:80]}...")
        print(f"    Reference: {sample['reference_text'][:80]}...")
        print(f"    Predicted: {sample['predicted_text'][:80]}...")
        print(f"    Ref Emotion: {sample['reference_emotion']}")
        print(f"    Pred Emotion: {sample['predicted_emotion']} (conf: {sample['emotion_confidence']:.3f})")
        print(f"    BLEU: {sample['bleu']:.3f}, ROUGE-1: {sample['rouge1']:.3f}")

if __name__ == "__main__":
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/test.json"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Load evaluation dataset
    eval_dataset = TherapyEvaluationDataset(TEST_DATA_PATH)

    print("\n=== BASELINE WITH FULL STRUCTURED INPUT ===")
    baseline_full_results = evaluate_baseline(eval_dataset, DEVICE, use_full_input=True, top_p=0.8)
    print_evaluation_results(baseline_full_results)


SFT Hyperparameters Tuning


In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import itertools
import pickle
import os
from datetime import datetime
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set seed at import
set_seed(42)

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Model Output Extraction ======================
def extract_model_output(response):
    """Extract therapist text and emotion from model output"""
    response = response.strip()

    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Find <therapist_emotion> tag
    emotion_pattern = r'<therapist_emotion>'
    emotion_match = re.search(emotion_pattern, response)

    if emotion_match:
        # Extract text before <therapist_emotion>
        therapist_text = response[:emotion_match.start()].strip()

        # Extract emotion part after <therapist_emotion>
        emotion_part = response[emotion_match.end():].strip()

        # Get the first word as emotion
        emotion_words = emotion_part.split()
        emotion = emotion_words[0].lower() if emotion_words else ""

        return therapist_text, emotion, True
    else:
        # No emotion tag found, return entire response as text
        return response, "", False

# ====================== Reference Data Processing ======================
class TherapyEvaluationDataset:
    """Dataset class for evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None, subset_size=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self.subset_size = subset_size
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation"""
        print(f"Processing evaluation data (subset_size: {self.subset_size})...")

        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input prompt
                    input_parts = []
                    if problem_type:
                        input_parts.append(f"<problem>{problem_type}")

                    input_parts.append(f"<user>{combined_user_text}")
                    if last_user_emotion:
                        input_parts.append(f"<user_emotion>{last_user_emotion}")

                    input_parts.append("<therapist>")
                    input_text = " ".join(input_parts)

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'reference_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

                    if self.subset_size and len(self.evaluation_data) >= self.subset_size:
                        break

            if self.subset_size and len(self.evaluation_data) >= self.subset_size:
                break

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Fast Evaluation Function ======================
def evaluate_hyperparameters(model, tokenizer, evaluation_dataset, device='cuda',
                            max_new_tokens=128, top_p=0.8, top_k=0, temperature=1.0, do_sample=True,
                            verbose=False):
    """
    Fast evaluation for hyperparameter tuning

    Args:
        model: Trained model
        tokenizer: Model tokenizer
        evaluation_dataset: TherapyEvaluationDataset instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        top_k: Top-k sampling parameter
        temperature: Temperature for sampling
        do_sample: Whether to use sampling
        verbose: Whether to print detailed progress

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
    }

    model.eval()

    # Progress bar only if verbose
    if verbose:
        progress_bar = tqdm(evaluation_data, desc=f"Eval p={top_p:.2f} k={top_k} T={temperature:.1f}")
        data_iter = progress_bar
    else:
        data_iter = evaluation_data

    for i, sample in enumerate(data_iter):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text, predicted_emotion, has_emotion = extract_model_output(response)

        # Compute metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower()) if has_emotion else False

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

    if verbose and 'progress_bar' in locals():
        progress_bar.close()

    # Compute summary statistics
    summary = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data),
        'top_p': top_p,
        'top_k': top_k,
        'temperature': temperature
    }

    return summary

# ====================== Model Loading Functions ======================
def load_sft_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading SFT checkpoint: {checkpoint_path}")

    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    return model, tokenizer

def load_rl_model(model_dir, tokenizer_path, device='cuda'):
    """Load RL model from directory"""
    from transformers import AutoModelForCausalLM

    print(f"Loading RL model from directory: {model_dir}")

    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

    return model, tokenizer

def load_model_auto(model_path, tokenizer_path, device='cuda'):
    """Automatically detect and load model"""
    import os

    if os.path.isfile(model_path):
        print("Detected SFT checkpoint file")
        model, tokenizer = load_sft_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'sft'
    elif os.path.isdir(model_path):
        print("Detected RL model directory")
        model, tokenizer = load_rl_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'rl'
    else:
        raise ValueError(f"Model path {model_path} is neither a file nor a directory")

# ====================== Hyperparameter Tuning Functions ======================
def compute_combined_score(results, weights=None):
    """
    Compute a combined score from multiple metrics

    Args:
        results: Dictionary containing metric results
        weights: Dictionary of weights for each metric

    Returns:
        Combined score (higher is better)
    """
    if weights is None:
        weights = {
            'avg_bleu': 0.2,
            'avg_rouge1': 0.15,
            'avg_rouge2': 0.15,
            'avg_rougeL': 0.15,
            'avg_meteor': 0.2,
            'emotion_accuracy': 0.1,
            'emotion_tag_coverage': 0.05
        }

    score = 0.0
    for metric, weight in weights.items():
        if metric in results:
            score += results[metric] * weight

    return score

def hyperparameter_search(model_path, tokenizer_path, test_data_path, device='cuda',
                         top_p_values=None, top_k_values=None, temperature_values=None, subset_size=100,
                         max_new_tokens=128, output_dir=None, weights=None):
    """
    Perform grid search over top_p, top_k, and temperature values

    Args:
        model_path: Path to trained model
        tokenizer_path: Path to tokenizer
        test_data_path: Path to test JSON data
        device: Device to run evaluation on
        top_p_values: List of top_p values to test
        top_k_values: List of top_k values to test
        temperature_values: List of temperature values to test
        subset_size: Number of samples to use for evaluation
        max_new_tokens: Maximum tokens to generate
        output_dir: Directory to save results
        weights: Weights for combining metrics

    Returns:
        Dictionary with all results and best parameters
    """

    # Default hyperparameter values
    if top_p_values is None:
        top_p_values = [0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

    if top_k_values is None:
        top_k_values = [0, 5, 10, 20, 40, 50]  # 0 means no top_k filtering

    if temperature_values is None:
        temperature_values = [0.8, 1.0, 1.1, 1.2, 1.3, 1.4]

    # Load model and tokenizer
    print(f"Loading model from {model_path}...")
    model, tokenizer, model_type = load_model_auto(model_path, tokenizer_path, device)
    print(f"Loaded {model_type.upper()} model successfully")

    # Load evaluation dataset (subset)
    print(f"Loading evaluation data subset (size: {subset_size})...")
    eval_dataset = TherapyEvaluationDataset(test_data_path, tokenizer_path, subset_size=subset_size)

    # Initialize results storage
    all_results = []
    best_score = -1
    best_params = None

    # Total combinations
    total_combinations = len(top_p_values) * len(top_k_values) * len(temperature_values)
    print(f"Testing {total_combinations} hyperparameter combinations...")
    print(f"top_p values: {top_p_values}")
    print(f"top_k values: {top_k_values}")
    print(f"temperature values: {temperature_values}")
    print(f"Subset size: {subset_size} samples")

    # Grid search
    combination_count = 0

    for top_p in top_p_values:
        for top_k in top_k_values:
            for temperature in temperature_values:
                combination_count += 1

                print(f"\n[{combination_count}/{total_combinations}] Testing top_p={top_p}, top_k={top_k}, temp={temperature}")

                # Evaluate with current hyperparameters
                results = evaluate_hyperparameters(
                    model, tokenizer, eval_dataset, device,
                    max_new_tokens=max_new_tokens,
                    top_p=top_p, top_k=top_k, temperature=temperature,
                    verbose=False
                )

                # Compute combined score
                combined_score = compute_combined_score(results, weights)
                results['combined_score'] = combined_score

                # Store results
                all_results.append(results)

                # Check if this is the best so far
                if combined_score > best_score:
                    best_score = combined_score
                    best_params = {'top_p': top_p, 'top_k': top_k, 'temperature': temperature}
                    print(f"  *** NEW BEST *** Score: {combined_score:.4f}")

                # Print current results
                print(f"  BLEU: {results['avg_bleu']:.4f}, "
                      f"ROUGE-1: {results['avg_rouge1']:.4f}, "
                      f"ROUGE-2: {results['avg_rouge2']:.4f}, "
                      f"ROUGE-L: {results['avg_rougeL']:.4f}, "
                      f"METEOR: {results['avg_meteor']:.4f}, "
                      f"Emotion: {results['emotion_accuracy']:.4f}, "
                      f"Combined: {combined_score:.4f}")

    # Compile final results
    final_results = {
        'best_params': best_params,
        'best_score': best_score,
        'all_results': all_results,
        'model_path': model_path,
        'model_type': model_type,
        'subset_size': subset_size,
        'search_space': {
            'top_p_values': top_p_values,
            'top_k_values': top_k_values,
            'temperature_values': temperature_values
        },
        'weights': weights or {
            'avg_bleu': 0.2,
            'avg_rouge1': 0.15,
            'avg_rouge2': 0.15,
            'avg_rougeL': 0.15,
            'avg_meteor': 0.2,
            'emotion_accuracy': 0.1,
            'emotion_tag_coverage': 0.05
        },
        'timestamp': datetime.now().isoformat()
    }

    # Print summary
    print(f"\n{'='*60}")
    print("HYPERPARAMETER SEARCH COMPLETED")
    print(f"{'='*60}")
    print(f"Best parameters: top_p={best_params['top_p']}, top_k={best_params['top_k']}, temperature={best_params['temperature']}")
    print(f"Best combined score: {best_score:.4f}")

    # Find best result details
    best_result = max(all_results, key=lambda x: x['combined_score'])
    print(f"\nBest result details:")
    print(f"  BLEU Score: {best_result['avg_bleu']:.4f}")
    print(f"  ROUGE-1: {best_result['avg_rouge1']:.4f}")
    print(f"  ROUGE-2: {best_result['avg_rouge2']:.4f}")
    print(f"  ROUGE-L: {best_result['avg_rougeL']:.4f}")
    print(f"  METEOR: {best_result['avg_meteor']:.4f}")
    print(f"  Emotion Accuracy: {best_result['emotion_accuracy']:.4f}")
    print(f"  Emotion Coverage: {best_result['emotion_tag_coverage']:.4f}")
    print(f"  Combined Score: {best_result['combined_score']:.4f}")

    # Save results if output directory specified
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = os.path.join(output_dir, f"hyperparameter_search_{timestamp}.pkl")

        with open(results_file, 'wb') as f:
            pickle.dump(final_results, f)

        print(f"\nResults saved to: {results_file}")

        # Also save a summary CSV
        import pandas as pd
        df_results = pd.DataFrame(all_results)
        csv_file = os.path.join(output_dir, f"hyperparameter_results_{timestamp}.csv")
        df_results.to_csv(csv_file, index=False)
        print(f"CSV summary saved to: {csv_file}")

    return final_results

def analyze_results(results_file):
    """Analyze hyperparameter search results"""
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

    import pandas as pd

    # Convert to DataFrame
    df = pd.DataFrame(results['all_results'])

    print(f"Hyperparameter Search Analysis")
    print(f"=" * 50)
    print(f"Model: {results['model_path']}")
    print(f"Search completed: {results['timestamp']}")
    print(f"Total combinations tested: {len(df)}")

    # Top 5 results
    print(f"\nTop 5 Results:")
    top_5 = df.nlargest(5, 'combined_score')[['top_p', 'top_k', 'temperature', 'avg_bleu', 'avg_rouge1',
                                              'avg_rouge2', 'avg_rougeL', 'avg_meteor',
                                              'emotion_accuracy', 'combined_score']]
    print(top_5.to_string(index=False))

    print(f"\nBest parameters for each metric:")
    for metric in ['avg_bleu', 'avg_rouge1', 'avg_rouge2', 'avg_rougeL', 'avg_meteor', 'emotion_accuracy']:
        best_row = df.loc[df[metric].idxmax()]
        print(f"  {metric}: top_p={best_row['top_p']}, top_k={best_row['top_k']}, temp={best_row['temperature']} (score: {best_row[metric]:.4f})")

    return df

# ====================== Usage Example ======================
if __name__ == "__main__":
    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt"
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/test.json"
    OUTPUT_DIR = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/Evaluation2/SFThyperparameter_search1"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Hyperparameter search space
    TOP_P_VALUES = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    TOP_K_VALUES = [0, 5, 10, 15, 20, 25, 30, 35]
    TEMPERATURE_VALUES = [1.0, 1.1, 1.2, 1.3]

    SUBSET_SIZE = 100

    CUSTOM_WEIGHTS = None

    # Run hyperparameter search
    results = hyperparameter_search(
        model_path=MODEL_PATH,
        tokenizer_path=TOKENIZER_PATH,
        test_data_path=TEST_DATA_PATH,
        device=DEVICE,
        top_p_values=TOP_P_VALUES,
        top_k_values=TOP_K_VALUES,
        temperature_values=TEMPERATURE_VALUES,
        subset_size=SUBSET_SIZE,
        output_dir=OUTPUT_DIR,
        weights=CUSTOM_WEIGHTS
    )

    print(f"\nBest hyperparameters found:")
    print(f"top_p: {results['best_params']['top_p']}")
    print(f"top_k: {results['best_params']['top_k']}")
    print(f"temperature: {results['best_params']['temperature']}")
    print(f"Score: {results['best_score']:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Loading model from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt...
Detected SFT checkpoint file
Loading SFT checkpoint: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Loaded SFT model successfully
Loading evaluation data subset (size: 100)...
Processing evaluation data (subset_size: 100)...


Processing evaluation data:  24%|██▎       | 24/102 [00:00<00:00, 5276.41it/s]

Processed 100 evaluation samples
Testing 256 hyperparameter combinations...
top_p values: [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
top_k values: [0, 5, 10, 15, 20, 25, 30, 35]
temperature values: [1.0, 1.1, 1.2, 1.3]
Subset size: 100 samples

[1/256] Testing top_p=0.6, top_k=0, temp=1.0


  *** NEW BEST *** Score: 0.2035
  BLEU: 0.0153, ROUGE-1: 0.1341, ROUGE-2: 0.0313, ROUGE-L: 0.1273, METEOR: 0.0627, Emotion: 0.9400, Combined: 0.2035

[2/256] Testing top_p=0.6, top_k=0, temp=1.1
  BLEU: 0.0148, ROUGE-1: 0.1206, ROUGE-2: 0.0162, ROUGE-L: 0.1143, METEOR: 0.0577, Emotion: 0.8900, Combined: 0.1906

[3/256] Testing top_p=0.6, top_k=0, temp=1.2
  BLEU: 0.0136, ROUGE-1: 0.1116, ROUGE-2: 0.0209, ROUGE-L: 0.1042, METEOR: 0.0576, Emotion: 0.8300, Combined: 0.1802

[4/256] Testing top_p=0.6, top_k=0, temp=1.3
  BLEU: 0.0078, ROUGE-1: 0.0800, ROUGE-2: 0.0017, ROUGE-L: 0.0761, METEOR: 0.0520, Emotion: 0.7700, Combined: 0.1571

[5/256] Testing top_p=0.6, top_k=5, temp=1.0
  BLEU: 0.0155, ROUGE-1: 0.1291, ROUGE-2: 0.0251, ROUGE-L: 0.1256, METEOR: 0.0593, Emotion: 0.9700, Combined: 0.2034

[6/256] Testing top_p=0.6, top_k=5, temp=1.1
  BLEU: 0.0130, ROUGE-1: 0.1354, ROUGE-2: 0.0187, ROUGE-L: 0.1275, METEOR: 0.0608, Emotion: 0.9300, Combined: 0.2000

[7/256] Testing top_p=0.6, top_k=5

SFT Evaluation

In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Model Format Detection ======================
def detect_model_format(model, tokenizer, device='cuda', test_inputs=None):
    """
    Test model output format (should be consistent since both SFT and RL use same format)

    Args:
        model: The model to test
        tokenizer: Model tokenizer
        device: Device to run test on
        test_inputs: List of test input strings, uses defaults if None

    Returns:
        str: 'standard' if using <therapist_emotion> format, 'unknown' otherwise
    """
    if test_inputs is None:
        test_inputs = [
            "<problem>anxiety <user>I'm worried about work <user_emotion>anxiety <therapist>",
            "<problem>depression <user>I feel very sad <user_emotion>sadness <therapist>",
            "<problem>relationship <user>My partner doesn't understand me <user_emotion>anger <therapist>"
        ]

    emotion_tag_count = 0
    total_tests = len(test_inputs)

    model.eval()
    with torch.no_grad():
        for test_input in test_inputs:
            # Tokenize and generate
            input_ids = tokenizer.encode(test_input, return_tensors='pt').to(device)
            outputs = model.generate(
                input_ids,
                max_new_tokens=128,
                do_sample=True,
                top_p=0.8,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

            # Decode response
            new_tokens = outputs[0][len(input_ids[0]):]
            response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

            # Check for emotion tag
            if '<therapist_emotion>' in response:
                emotion_tag_count += 1

    if emotion_tag_count >= total_tests * 0.5:
        return 'standard'
    else:
        return 'unknown'

# ====================== Model Output Extraction ======================
def extract_model_output(response):
    """
    Extract therapist text and emotion from model output.
    Both SFT and RL models use format: 'text <therapist_emotion> emotion<eos>'

    Returns:
        therapist_text (str): Text before <therapist_emotion>
        emotion (str): Emotion word after <therapist_emotion>
        has_emotion_tag (bool): Whether emotion tag was found
    """
    response = response.strip()

    # Remove <eos> if present
    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Find <therapist_emotion> tag
    emotion_pattern = r'<therapist_emotion>'
    emotion_match = re.search(emotion_pattern, response)

    if emotion_match:
        # Extract text before <therapist_emotion>
        therapist_text = response[:emotion_match.start()].strip()

        # Extract emotion part after <therapist_emotion>
        emotion_part = response[emotion_match.end():].strip()

        # Get the first word as emotion
        emotion_words = emotion_part.split()
        emotion = emotion_words[0].lower() if emotion_words else ""

        return therapist_text, emotion, True
    else:
        # No emotion tag found, return entire response as text
        return response, "", False

def extract_emotion_only(response):
    """Extract only emotion from response (for compatibility with existing code)"""
    _, emotion, has_emotion = extract_model_output(response)
    return emotion if has_emotion else None

# ====================== Reference Data Processing ======================
class TherapyEvaluationDataset:
    """Dataset class for evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            # Use default tokenizer if path not provided
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation"""
        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input prompt
                    input_parts = []
                    if problem_type:
                        input_parts.append(f"<problem>{problem_type}")

                    input_parts.append(f"<user>{combined_user_text}")
                    if last_user_emotion:
                        input_parts.append(f"<user_emotion>{last_user_emotion}")

                    input_parts.append("<therapist>")
                    input_text = " ".join(input_parts)

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'reference_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Model Evaluation ======================
def evaluate_model(model, tokenizer, evaluation_dataset, device='cuda', max_new_tokens=128,
                  top_p=0.6, top_k=30, temperature=1.0, do_sample=True):
    """
    Evaluate model on therapy dataset

    Args:
        model: Trained model
        tokenizer: Model tokenizer
        evaluation_dataset: TherapyEvaluationDataset instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        do_sample: Whether to use sampling

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
        'sample_outputs': []
    }

    emotion_confusion = defaultdict(lambda: defaultdict(int))

    model.eval()

    # Detect model output format
    detected_format = detect_model_format(model, tokenizer, device)
    print(f"Output format check: {detected_format.upper()}")

    print(f"Evaluating model on {len(evaluation_data)} samples...")

    progress_bar = tqdm(evaluation_data, desc="Evaluating",
                       bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')

    for i, sample in enumerate(progress_bar):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text, predicted_emotion, has_emotion = extract_model_output(response)

        # Compute text generation metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower()) if has_emotion else False

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

        if has_emotion:
            emotion_confusion[reference_emotion.lower()][predicted_emotion.lower()] += 1
        else:
            emotion_confusion[reference_emotion.lower()]['no_prediction'] += 1

        if i < 10:
            results['sample_outputs'].append({
                'input': input_text,
                'reference_text': reference_text,
                'predicted_text': predicted_text,
                'reference_emotion': reference_emotion,
                'predicted_emotion': predicted_emotion,
                'full_response': response,
                'bleu': bleu,
                'rouge1': rouge['rouge1'],
                'meteor': meteor
            })

        if (i + 1) % 20 == 0 or i < 5:
            sample_info = f"Sample {i+1}: BLEU={bleu:.3f}, Emotion={'✓' if emotion_correct else '✗'}"
            progress_bar.set_description(f"Evaluating - {sample_info}")

            if (i + 1) % 50 == 0 or i < 3:
                print(f"\n" + "="*80)
                print(f"LIVE SAMPLE {i+1}/{len(evaluation_data)}")
                print("="*80)
                print(f"Input: {input_text[:120]}...")
                print(f"Reference: {reference_text[:100]}...")
                print(f"Generated: {predicted_text[:100]}...")
                print(f"Full Response: {response[:150]}...")
                print(f"Ref Emotion: {reference_emotion} | Pred Emotion: {predicted_emotion} | Match: {'✓' if emotion_correct else '✗'}")
                print(f"BLEU: {bleu:.4f} | ROUGE-1: {rouge['rouge1']:.4f} | METEOR: {meteor:.4f}")
                print("="*80)
        else:
            if i > 0:
                avg_bleu = np.mean(results['bleu_scores'])
                avg_emotion_acc = np.mean(results['emotion_accuracy'])
                progress_bar.set_description(f"Evaluating - Avg BLEU: {avg_bleu:.3f}, Emotion Acc: {avg_emotion_acc:.3f}")

    progress_bar.close()

    # Compute summary statistics
    results['summary'] = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data)
    }

    results['emotion_confusion_matrix'] = dict(emotion_confusion)
    results['detected_format'] = detected_format

    # Print final progress summary
    print(f"\n{'='*60}")
    print(f"EVALUATION COMPLETED!")
    print(f"{'='*60}")
    print(f"Processed {len(evaluation_data)} samples")
    print(f"Average BLEU: {results['summary']['avg_bleu']:.4f}")
    print(f"Average ROUGE-1: {results['summary']['avg_rouge1']:.4f}")
    print(f"Emotion Accuracy: {results['summary']['emotion_accuracy']:.4f}")
    print(f"Emotion Tag Coverage: {results['summary']['emotion_tag_coverage']:.4f}")
    print(f"{'='*60}")

    return results

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    summary = results['summary']

    print("\n" + "="*60)
    print("THERAPY MODEL EVALUATION RESULTS")
    print("="*60)
    print(f"Model Type: {results.get('model_type', 'Unknown').upper()}")
    print(f"Output Format: {results.get('detected_format', 'Unknown').upper()}")

    print(f"\nText Generation Metrics:")
    print(f"  BLEU Score:     {summary['avg_bleu']:.4f}")
    print(f"  ROUGE-1:        {summary['avg_rouge1']:.4f}")
    print(f"  ROUGE-2:        {summary['avg_rouge2']:.4f}")
    print(f"  ROUGE-L:        {summary['avg_rougeL']:.4f}")
    print(f"  METEOR:         {summary['avg_meteor']:.4f}")

    print(f"\nEmotion Prediction:")
    print(f"  Emotion Accuracy:    {summary['emotion_accuracy']:.4f}")
    print(f"  Emotion Tag Coverage: {summary['emotion_tag_coverage']:.4f}")

    print(f"\nDataset Info:")
    print(f"  Total Samples:  {summary['total_samples']}")

    print(f"\nSample Outputs:")
    for i, sample in enumerate(results['sample_outputs'][:3], 1):
        print(f"\n  Sample {i}:")
        print(f"    Input: {sample['input'][:100]}...")
        print(f"    Reference: {sample['reference_text'][:80]}...")
        print(f"    Predicted: {sample['predicted_text'][:80]}...")
        print(f"    Ref Emotion: {sample['reference_emotion']}")
        print(f"    Pred Emotion: {sample['predicted_emotion']}")
        print(f"    BLEU: {sample['bleu']:.3f}, ROUGE-1: {sample['rouge1']:.3f}")

# ====================== Model Loading Functions ======================
def load_sft_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading SFT checkpoint: {checkpoint_path}")

    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    if 'epoch' in checkpoint:
        print(f"Checkpoint info: Epoch {checkpoint['epoch']}, Loss {checkpoint.get('valid_loss', 'N/A')}")

    return model, tokenizer

def load_rl_model(model_dir, tokenizer_path, device='cuda'):
    """Load RL model from directory"""
    from transformers import AutoModelForCausalLM

    print(f"Loading RL model from directory: {model_dir}")

    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

    return model, tokenizer

def load_model_auto(model_path, tokenizer_path, device='cuda'):
    """
    Automatically detect and load model (SFT checkpoint or RL directory)

    Args:
        model_path: Path to model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        device: Device to load model on

    Returns:
        tuple: (model, tokenizer, model_type)
    """
    import os

    if os.path.isfile(model_path):
        print("Detected SFT checkpoint file")
        model, tokenizer = load_sft_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'sft'

    elif os.path.isdir(model_path):
        print("Detected RL model directory")
        model, tokenizer = load_rl_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'rl'

    else:
        raise ValueError(f"Model path {model_path} is neither a file nor a directory")

# ====================== Main Evaluation Function ======================
def run_evaluation(model_path, tokenizer_path, test_data_path, device='cuda'):
    """
    Run complete evaluation pipeline

    Args:
        model_path: Path to trained model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        test_data_path: Path to test JSON data
        device: Device to run evaluation on
    """

    # Load model and tokenizer (auto-detect type)
    print(f"Loading model from {model_path}...")
    model, tokenizer, model_type = load_model_auto(model_path, tokenizer_path, device)
    print(f"Loaded {model_type.upper()} model successfully")

    # Load evaluation dataset
    print(f"Loading evaluation data from {test_data_path}...")
    eval_dataset = TherapyEvaluationDataset(test_data_path, tokenizer_path)

    # Run evaluation
    results = evaluate_model(model, tokenizer, eval_dataset, device=device)
    results['model_type'] = model_type

    print_evaluation_results(results)

    return results

if __name__ == "__main__":


    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt"  # Will auto-detect type
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/test.json"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    results = run_evaluation(MODEL_PATH, TOKENIZER_PATH, TEST_DATA_PATH, DEVICE)

    import pickle
    with open('/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/Evaluation2/0.6SFT_evaluation_results.pkl', 'wb') as f:
      pickle.dump(results, f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Loading model from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt...
Detected SFT checkpoint file
Loading SFT checkpoint: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...


LIVE SAMPLE 300/454
Input: <problem>Self-punishment and disgust <user>I wanted Natalie up in the air, as far away from this as possible. Connie was...
Reference: I think I heard my father use that expression....
Generated: I don't know....
Full Response: I don't know. <therapist_emotion> neutral<eos>...
Ref Emotion: neutral | Pred Emotion: neutral | Match: ✓
BLEU: 0.0154 | ROUGE-1: 0.1538 | METEOR: 0.0595


Evaluating - Avg BLEU: 0.011, Emotion Acc: 0.950:  88%|████████▊ | 400/454 [00:50<00:07,  7.56it/s]


LIVE SAMPLE 400/454
Input: <problem>The relationship with friends and family <user>But you know what could be helpful, actually? If you could just ...
Reference: Isn't this what you used to do with Russell?...
Generated: Would you like me to write something about you?...
Full Response: Would you like me to write something about you? <therapist_emotion> neutral<eos>...
Ref Emotion: neutral | Pred Emotion: neutral | Match: ✓
BLEU: 0.0285 | ROUGE-1: 0.2105 | METEOR: 0.1111


Evaluating - Avg BLEU: 0.011, Emotion Acc: 0.954: 100%|██████████| 454/454 [00:56<00:00,  8.08it/s]



EVALUATION COMPLETED!
Processed 454 samples
Average BLEU: 0.0108
Average ROUGE-1: 0.1164
Emotion Accuracy: 0.9537
Emotion Tag Coverage: 1.0000

THERAPY MODEL EVALUATION RESULTS
Model Type: SFT
Output Format: STANDARD

Text Generation Metrics:
  BLEU Score:     0.0108
  ROUGE-1:        0.1164
  ROUGE-2:        0.0175
  ROUGE-L:        0.1076
  METEOR:         0.0485

Emotion Prediction:
  Emotion Accuracy:    0.9537
  Emotion Tag Coverage: 1.0000

Dataset Info:
  Total Samples:  454

Sample Outputs:

  Sample 1:
    Input: <problem>Breakups or Divorce <user>I mean, shit, don't you know that men are the new women? Obsessed...
    Reference: But I thought you said...
    Predicted: What is it?...
    Ref Emotion: neutral
    Pred Emotion: neutral
    BLEU: 0.000, ROUGE-1: 0.000

  Sample 2:
    Input: <problem>Breakups or Divorce <user>Well, we were watching tv.But before that we were having dinner a...
    Reference: So you were the one who came up with it....
    Predicted: Well, maybe

RL Hyperparameters


In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import itertools
import pickle
import os
from datetime import datetime
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Model Output Extraction ======================
def extract_model_output(response):
    """Extract therapist text and emotion from model output"""
    response = response.strip()

    # Remove <eos> if present
    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Find <therapist_emotion> tag
    emotion_pattern = r'<therapist_emotion>'
    emotion_match = re.search(emotion_pattern, response)

    if emotion_match:
        # Extract text before <therapist_emotion>
        therapist_text = response[:emotion_match.start()].strip()

        # Extract emotion part after <therapist_emotion>
        emotion_part = response[emotion_match.end():].strip()

        # Get the first word as emotion
        emotion_words = emotion_part.split()
        emotion = emotion_words[0].lower() if emotion_words else ""

        return therapist_text, emotion, True
    else:
        # No emotion tag found, return entire response as text
        return response, "", False

# ====================== Reference Data Processing ======================
class TherapyEvaluationDataset:
    """Dataset class for evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None, subset_size=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self.subset_size = subset_size
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation"""
        print(f"Processing evaluation data (subset_size: {self.subset_size})...")

        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input prompt
                    input_parts = []
                    if problem_type:
                        input_parts.append(f"<problem>{problem_type}")

                    input_parts.append(f"<user>{combined_user_text}")
                    if last_user_emotion:
                        input_parts.append(f"<user_emotion>{last_user_emotion}")

                    input_parts.append("<therapist>")
                    input_text = " ".join(input_parts)

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'reference_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

                    # Early stopping if subset size is reached
                    if self.subset_size and len(self.evaluation_data) >= self.subset_size:
                        break

            # Early stopping if subset size is reached
            if self.subset_size and len(self.evaluation_data) >= self.subset_size:
                break

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ======================  Evaluation Function ======================
def evaluate_hyperparameters(model, tokenizer, evaluation_dataset, device='cuda',
                            max_new_tokens=128, top_p=0.8, top_k=0, temperature=1.0, do_sample=True,
                            verbose=False):
    """
    Fast evaluation for hyperparameter tuning

    Args:
        model: Trained model
        tokenizer: Model tokenizer
        evaluation_dataset: TherapyEvaluationDataset instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        top_k: Top-k sampling parameter
        temperature: Temperature for sampling
        do_sample: Whether to use sampling
        verbose: Whether to print detailed progress

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
    }

    model.eval()

    # Progress bar only if verbose
    if verbose:
        progress_bar = tqdm(evaluation_data, desc=f"Eval p={top_p:.2f} k={top_k} T={temperature:.1f}")
        data_iter = progress_bar
    else:
        data_iter = evaluation_data

    for i, sample in enumerate(data_iter):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text, predicted_emotion, has_emotion = extract_model_output(response)

        # Compute metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower()) if has_emotion else False

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

    if verbose and 'progress_bar' in locals():
        progress_bar.close()

    # Compute summary statistics
    summary = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data),
        'top_p': top_p,
        'top_k': top_k,
        'temperature': temperature
    }

    return summary

# ====================== Model Loading Functions ======================
def load_sft_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading SFT checkpoint: {checkpoint_path}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    # Initialize model
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    return model, tokenizer

def load_rl_model(model_dir, tokenizer_path, device='cuda'):
    """Load RL model from directory"""
    from transformers import AutoModelForCausalLM

    print(f"Loading RL model from directory: {model_dir}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load model from directory
    model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

    return model, tokenizer

def load_model_auto(model_path, tokenizer_path, device='cuda'):
    """Automatically detect and load model"""
    import os

    if os.path.isfile(model_path):
        print("Detected SFT checkpoint file")
        model, tokenizer = load_sft_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'sft'
    elif os.path.isdir(model_path):
        print("Detected RL model directory")
        model, tokenizer = load_rl_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'rl'
    else:
        raise ValueError(f"Model path {model_path} is neither a file nor a directory")

# ====================== Hyperparameter Tuning Functions ======================
def compute_combined_score(results, weights=None):
    """
    Compute a combined score from multiple metrics

    Args:
        results: Dictionary containing metric results
        weights: Dictionary of weights for each metric

    Returns:
        Combined score (higher is better)
    """
    if weights is None:
        # Default weights - adjust based on importance
        weights = {
            'avg_bleu': 0.2,
            'avg_rouge1': 0.15,
            'avg_rouge2': 0.15,
            'avg_rougeL': 0.15,
            'avg_meteor': 0.2,
            'emotion_accuracy': 0.1,
            'emotion_tag_coverage': 0.05
        }

    score = 0.0
    for metric, weight in weights.items():
        if metric in results:
            score += results[metric] * weight

    return score

def hyperparameter_search(model_path, tokenizer_path, test_data_path, device='cuda',
                         top_p_values=None, top_k_values=None, temperature_values=None, subset_size=100,
                         max_new_tokens=128, output_dir=None, weights=None):
    """
    Perform grid search over top_p, top_k, and temperature values

    Args:
        model_path: Path to trained model
        tokenizer_path: Path to tokenizer
        test_data_path: Path to test JSON data
        device: Device to run evaluation on
        top_p_values: List of top_p values to test
        top_k_values: List of top_k values to test
        temperature_values: List of temperature values to test
        subset_size: Number of samples to use for evaluation
        max_new_tokens: Maximum tokens to generate
        output_dir: Directory to save results
        weights: Weights for combining metrics

    Returns:
        Dictionary with all results and best parameters
    """

    # Default hyperparameter values
    if top_p_values is None:
        top_p_values = [0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

    if top_k_values is None:
        top_k_values = [0, 5, 10, 20, 40, 50]  # 0 means no top_k filtering

    if temperature_values is None:
        temperature_values = [0.8, 1.0, 1.1, 1.2, 1.3, 1.4]

    # Load model and tokenizer
    print(f"Loading model from {model_path}...")
    model, tokenizer, model_type = load_model_auto(model_path, tokenizer_path, device)
    print(f"Loaded {model_type.upper()} model successfully")

    # Load evaluation dataset (subset)
    print(f"Loading evaluation data subset (size: {subset_size})...")
    eval_dataset = TherapyEvaluationDataset(test_data_path, tokenizer_path, subset_size=subset_size)

    # Initialize results storage
    all_results = []
    best_score = -1
    best_params = None

    # Total combinations
    total_combinations = len(top_p_values) * len(top_k_values) * len(temperature_values)
    print(f"Testing {total_combinations} hyperparameter combinations...")
    print(f"top_p values: {top_p_values}")
    print(f"top_k values: {top_k_values}")
    print(f"temperature values: {temperature_values}")
    print(f"Subset size: {subset_size} samples")

    # Grid search
    combination_count = 0

    for top_p in top_p_values:
        for top_k in top_k_values:
            for temperature in temperature_values:
                combination_count += 1

                print(f"\n[{combination_count}/{total_combinations}] Testing top_p={top_p}, top_k={top_k}, temp={temperature}")

                # Evaluate with current hyperparameters
                results = evaluate_hyperparameters(
                    model, tokenizer, eval_dataset, device,
                    max_new_tokens=max_new_tokens,
                    top_p=top_p, top_k=top_k, temperature=temperature,
                    verbose=False
                )

                # Compute combined score
                combined_score = compute_combined_score(results, weights)
                results['combined_score'] = combined_score

                # Store results
                all_results.append(results)

                # Check if this is the best so far
                if combined_score > best_score:
                    best_score = combined_score
                    best_params = {'top_p': top_p, 'top_k': top_k, 'temperature': temperature}
                    print(f"  *** NEW BEST *** Score: {combined_score:.4f}")

                # Print current results
                print(f"  BLEU: {results['avg_bleu']:.4f}, "
                      f"ROUGE-1: {results['avg_rouge1']:.4f}, "
                      f"ROUGE-2: {results['avg_rouge2']:.4f}, "
                      f"ROUGE-L: {results['avg_rougeL']:.4f}, "
                      f"METEOR: {results['avg_meteor']:.4f}, "
                      f"Emotion: {results['emotion_accuracy']:.4f}, "
                      f"Combined: {combined_score:.4f}")

    # Compile final results
    final_results = {
        'best_params': best_params,
        'best_score': best_score,
        'all_results': all_results,
        'model_path': model_path,
        'model_type': model_type,
        'subset_size': subset_size,
        'search_space': {
            'top_p_values': top_p_values,
            'top_k_values': top_k_values,
            'temperature_values': temperature_values
        },
        'weights': weights or {
            'avg_bleu': 0.2,
            'avg_rouge1': 0.15,
            'avg_rouge2': 0.15,
            'avg_rougeL': 0.15,
            'avg_meteor': 0.2,
            'emotion_accuracy': 0.1,
            'emotion_tag_coverage': 0.05
        },
        'timestamp': datetime.now().isoformat()
    }

    # Print summary
    print(f"\n{'='*60}")
    print("HYPERPARAMETER SEARCH COMPLETED")
    print(f"{'='*60}")
    print(f"Best parameters: top_p={best_params['top_p']}, top_k={best_params['top_k']}, temperature={best_params['temperature']}")
    print(f"Best combined score: {best_score:.4f}")

    # Find best result details
    best_result = max(all_results, key=lambda x: x['combined_score'])
    print(f"\nBest result details:")
    print(f"  BLEU Score: {best_result['avg_bleu']:.4f}")
    print(f"  ROUGE-1: {best_result['avg_rouge1']:.4f}")
    print(f"  ROUGE-2: {best_result['avg_rouge2']:.4f}")
    print(f"  ROUGE-L: {best_result['avg_rougeL']:.4f}")
    print(f"  METEOR: {best_result['avg_meteor']:.4f}")
    print(f"  Emotion Accuracy: {best_result['emotion_accuracy']:.4f}")
    print(f"  Emotion Coverage: {best_result['emotion_tag_coverage']:.4f}")
    print(f"  Combined Score: {best_result['combined_score']:.4f}")

    # Save results if output directory specified
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = os.path.join(output_dir, f"hyperparameter_search_{timestamp}.pkl")

        with open(results_file, 'wb') as f:
            pickle.dump(final_results, f)

        print(f"\nResults saved to: {results_file}")

        # Also save a summary CSV
        import pandas as pd
        df_results = pd.DataFrame(all_results)
        csv_file = os.path.join(output_dir, f"hyperparameter_results_{timestamp}.csv")
        df_results.to_csv(csv_file, index=False)
        print(f"CSV summary saved to: {csv_file}")

    return final_results

def analyze_results(results_file):
    """Analyze hyperparameter search results"""
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

    import pandas as pd

    # Convert to DataFrame
    df = pd.DataFrame(results['all_results'])

    print(f"Hyperparameter Search Analysis")
    print(f"=" * 50)
    print(f"Model: {results['model_path']}")
    print(f"Search completed: {results['timestamp']}")
    print(f"Total combinations tested: {len(df)}")

    # Top 5 results
    print(f"\nTop 5 Results:")
    top_5 = df.nlargest(5, 'combined_score')[['top_p', 'top_k', 'temperature', 'avg_bleu', 'avg_rouge1',
                                              'avg_rouge2', 'avg_rougeL', 'avg_meteor',
                                              'emotion_accuracy', 'combined_score']]
    print(top_5.to_string(index=False))

    # Best for each metric
    print(f"\nBest parameters for each metric:")
    for metric in ['avg_bleu', 'avg_rouge1', 'avg_rouge2', 'avg_rougeL', 'avg_meteor', 'emotion_accuracy']:
        best_row = df.loc[df[metric].idxmax()]
        print(f"  {metric}: top_p={best_row['top_p']}, top_k={best_row['top_k']}, temp={best_row['temperature']} (score: {best_row[metric]:.4f})")

    return df

if __name__ == "__main__":
    # Configuration
    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4"
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/test.json"
    OUTPUT_DIR = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/Evaluation2/RLhyperparameter_search1"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Hyperparameter search space
    TOP_P_VALUES = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.00]
    TOP_K_VALUES = [0, 5, 10, 15, 20, 25, 30, 35]
    TEMPERATURE_VALUES = [1.0, 1.1, 1.2, 1.3]

    SUBSET_SIZE = 100

    CUSTOM_WEIGHTS = None

    results = hyperparameter_search(
        model_path=MODEL_PATH,
        tokenizer_path=TOKENIZER_PATH,
        test_data_path=TEST_DATA_PATH,
        device=DEVICE,
        top_p_values=TOP_P_VALUES,
        top_k_values=TOP_K_VALUES,
        temperature_values=TEMPERATURE_VALUES,
        subset_size=SUBSET_SIZE,
        output_dir=OUTPUT_DIR,
        weights=CUSTOM_WEIGHTS
    )

    print(f"\nBest hyperparameters found:")
    print(f"top_p: {results['best_params']['top_p']}")
    print(f"top_k: {results['best_params']['top_k']}")
    print(f"temperature: {results['best_params']['temperature']}")
    print(f"Score: {results['best_score']:.4f}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading model from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4...
Detected RL model directory
Loading RL model from directory: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4


Some weights of the model checkpoint at /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4 were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded RL model successfully
Loading evaluation data subset (size: 100)...
Processing evaluation data (subset_size: 100)...


Processing evaluation data:  24%|██▎       | 24/102 [00:00<00:00, 6780.50it/s]

Processed 100 evaluation samples
Testing 288 hyperparameter combinations...
top_p values: [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
top_k values: [0, 5, 10, 15, 20, 25, 30, 35]
temperature values: [1.0, 1.1, 1.2, 1.3]
Subset size: 100 samples

[1/288] Testing top_p=0.6, top_k=0, temp=1.0


  *** NEW BEST *** Score: 0.1965
  BLEU: 0.0084, ROUGE-1: 0.1265, ROUGE-2: 0.0166, ROUGE-L: 0.1221, METEOR: 0.0401, Emotion: 0.9700, Combined: 0.1965

[2/288] Testing top_p=0.6, top_k=0, temp=1.1
  *** NEW BEST *** Score: 0.1976
  BLEU: 0.0075, ROUGE-1: 0.1289, ROUGE-2: 0.0121, ROUGE-L: 0.1224, METEOR: 0.0429, Emotion: 0.9800, Combined: 0.1976

[3/288] Testing top_p=0.6, top_k=0, temp=1.2
  *** NEW BEST *** Score: 0.2052
  BLEU: 0.0144, ROUGE-1: 0.1344, ROUGE-2: 0.0189, ROUGE-L: 0.1256, METEOR: 0.0622, Emotion: 0.9800, Combined: 0.2052

[4/288] Testing top_p=0.6, top_k=0, temp=1.3
  BLEU: 0.0116, ROUGE-1: 0.1326, ROUGE-2: 0.0148, ROUGE-L: 0.1269, METEOR: 0.0570, Emotion: 0.9600, Combined: 0.2009

[5/288] Testing top_p=0.6, top_k=5, temp=1.0
  BLEU: 0.0114, ROUGE-1: 0.1402, ROUGE-2: 0.0211, ROUGE-L: 0.1321, METEOR: 0.0502, Emotion: 0.9700, Combined: 0.2023

[6/288] Testing top_p=0.6, top_k=5, temp=1.1
  BLEU: 0.0095, ROUGE-1: 0.1227, ROUGE-2: 0.0119, ROUGE-L: 0.1176, METEOR: 0.0378, Emo

RL Evaluation

In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Model Format Detection ======================
def detect_model_format(model, tokenizer, device='cuda', test_inputs=None):
    """
    Test model output format (should be consistent since both SFT and RL use same format)

    Args:
        model: The model to test
        tokenizer: Model tokenizer
        device: Device to run test on
        test_inputs: List of test input strings, uses defaults if None

    Returns:
        str: 'standard' if using <therapist_emotion> format, 'unknown' otherwise
    """
    if test_inputs is None:
        test_inputs = [
            "<problem>anxiety <user>I'm worried about work <user_emotion>anxiety <therapist>",
            "<problem>depression <user>I feel very sad <user_emotion>sadness <therapist>",
            "<problem>relationship <user>My partner doesn't understand me <user_emotion>anger <therapist>"
        ]

    emotion_tag_count = 0
    total_tests = len(test_inputs)

    model.eval()
    with torch.no_grad():
        for test_input in test_inputs:
            # Tokenize and generate
            input_ids = tokenizer.encode(test_input, return_tensors='pt').to(device)
            outputs = model.generate(
                input_ids,
                max_new_tokens=128,
                do_sample=True,
                top_p=0.8,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

            # Decode response
            new_tokens = outputs[0][len(input_ids[0]):]
            response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

            # Check for emotion tag
            if '<therapist_emotion>' in response:
                emotion_tag_count += 1

    # Determine format
    if emotion_tag_count >= total_tests * 0.5:
        return 'standard'
    else:
        return 'unknown'

# ====================== Model Output Extraction ======================
def extract_model_output(response):
    """
    Extract therapist text and emotion from model output.
    Both SFT and RL models use format: 'text <therapist_emotion> emotion<eos>'

    Returns:
        therapist_text (str): Text before <therapist_emotion>
        emotion (str): Emotion word after <therapist_emotion>
        has_emotion_tag (bool): Whether emotion tag was found
    """
    response = response.strip()

    # Remove <eos> if present
    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Find <therapist_emotion> tag
    emotion_pattern = r'<therapist_emotion>'
    emotion_match = re.search(emotion_pattern, response)

    if emotion_match:
        # Extract text before <therapist_emotion>
        therapist_text = response[:emotion_match.start()].strip()

        # Extract emotion part after <therapist_emotion>
        emotion_part = response[emotion_match.end():].strip()

        # Get the first word as emotion
        emotion_words = emotion_part.split()
        emotion = emotion_words[0].lower() if emotion_words else ""

        return therapist_text, emotion, True
    else:
        # No emotion tag found, return entire response as text
        return response, "", False

def extract_emotion_only(response):
    """Extract only emotion from response (for compatibility with existing code)"""
    _, emotion, has_emotion = extract_model_output(response)
    return emotion if has_emotion else None

# ====================== Reference Data Processing ======================
class TherapyEvaluationDataset:
    """Dataset class for evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation"""
        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input prompt
                    input_parts = []
                    if problem_type:
                        input_parts.append(f"<problem>{problem_type}")

                    input_parts.append(f"<user>{combined_user_text}")
                    if last_user_emotion:
                        input_parts.append(f"<user_emotion>{last_user_emotion}")

                    input_parts.append("<therapist>")
                    input_text = " ".join(input_parts)

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'reference_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Model Evaluation ======================
def evaluate_model(model, tokenizer, evaluation_dataset, device='cuda', max_new_tokens=128,
                  top_p=0.7, top_k=10, temperature=1.2, do_sample=True):
    """
    Evaluate model on therapy dataset

    Args:
        model: Trained model
        tokenizer: Model tokenizer
        evaluation_dataset: TherapyEvaluationDataset instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        do_sample: Whether to use sampling

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
        'sample_outputs': []
    }

    emotion_confusion = defaultdict(lambda: defaultdict(int))

    model.eval()

    # Detect model output format
    detected_format = detect_model_format(model, tokenizer, device)
    print(f"Output format check: {detected_format.upper()}")

    print(f"Evaluating model on {len(evaluation_data)} samples...")

    progress_bar = tqdm(evaluation_data, desc="Evaluating",
                       bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')

    for i, sample in enumerate(progress_bar):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text, predicted_emotion, has_emotion = extract_model_output(response)

        # Compute text generation metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower()) if has_emotion else False

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

        if has_emotion:
            emotion_confusion[reference_emotion.lower()][predicted_emotion.lower()] += 1
        else:
            emotion_confusion[reference_emotion.lower()]['no_prediction'] += 1

        # Store sample outputs for inspection
        if i < 10:
            results['sample_outputs'].append({
                'input': input_text,
                'reference_text': reference_text,
                'predicted_text': predicted_text,
                'reference_emotion': reference_emotion,
                'predicted_emotion': predicted_emotion,
                'full_response': response,
                'bleu': bleu,
                'rouge1': rouge['rouge1'],
                'meteor': meteor
            })

        if (i + 1) % 20 == 0 or i < 5:
            sample_info = f"Sample {i+1}: BLEU={bleu:.3f}, Emotion={'✓' if emotion_correct else '✗'}"
            progress_bar.set_description(f"Evaluating - {sample_info}")

            if (i + 1) % 50 == 0 or i < 3:
                print(f"\n" + "="*80)
                print(f"LIVE SAMPLE {i+1}/{len(evaluation_data)}")
                print("="*80)
                print(f"Input: {input_text[:120]}...")
                print(f"Reference: {reference_text[:100]}...")
                print(f"Generated: {predicted_text[:100]}...")
                print(f"Full Response: {response[:150]}...")
                print(f"Ref Emotion: {reference_emotion} | Pred Emotion: {predicted_emotion} | Match: {'✓' if emotion_correct else '✗'}")
                print(f"BLEU: {bleu:.4f} | ROUGE-1: {rouge['rouge1']:.4f} | METEOR: {meteor:.4f}")
                print("="*80)
        else:
            if i > 0:
                avg_bleu = np.mean(results['bleu_scores'])
                avg_emotion_acc = np.mean(results['emotion_accuracy'])
                progress_bar.set_description(f"Evaluating - Avg BLEU: {avg_bleu:.3f}, Emotion Acc: {avg_emotion_acc:.3f}")

    progress_bar.close()

    # Compute summary statistics
    results['summary'] = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data)
    }

    results['emotion_confusion_matrix'] = dict(emotion_confusion)
    results['detected_format'] = detected_format

    # Print final progress summary
    print(f"\n{'='*60}")
    print(f"EVALUATION COMPLETED!")
    print(f"{'='*60}")
    print(f"Processed {len(evaluation_data)} samples")
    print(f"Average BLEU: {results['summary']['avg_bleu']:.4f}")
    print(f"Average ROUGE-1: {results['summary']['avg_rouge1']:.4f}")
    print(f"Emotion Accuracy: {results['summary']['emotion_accuracy']:.4f}")
    print(f"Emotion Tag Coverage: {results['summary']['emotion_tag_coverage']:.4f}")
    print(f"{'='*60}")

    return results

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    summary = results['summary']

    print("\n" + "="*60)
    print("THERAPY MODEL EVALUATION RESULTS")
    print("="*60)
    print(f"Model Type: {results.get('model_type', 'Unknown').upper()}")
    print(f"Output Format: {results.get('detected_format', 'Unknown').upper()}")

    print(f"\nText Generation Metrics:")
    print(f"  BLEU Score:     {summary['avg_bleu']:.4f}")
    print(f"  ROUGE-1:        {summary['avg_rouge1']:.4f}")
    print(f"  ROUGE-2:        {summary['avg_rouge2']:.4f}")
    print(f"  ROUGE-L:        {summary['avg_rougeL']:.4f}")
    print(f"  METEOR:         {summary['avg_meteor']:.4f}")

    print(f"\nEmotion Prediction:")
    print(f"  Emotion Accuracy:    {summary['emotion_accuracy']:.4f}")
    print(f"  Emotion Tag Coverage: {summary['emotion_tag_coverage']:.4f}")

    print(f"\nDataset Info:")
    print(f"  Total Samples:  {summary['total_samples']}")

    print(f"\nSample Outputs:")
    for i, sample in enumerate(results['sample_outputs'][:3], 1):
        print(f"\n  Sample {i}:")
        print(f"    Input: {sample['input'][:100]}...")
        print(f"    Reference: {sample['reference_text'][:80]}...")
        print(f"    Predicted: {sample['predicted_text'][:80]}...")
        print(f"    Ref Emotion: {sample['reference_emotion']}")
        print(f"    Pred Emotion: {sample['predicted_emotion']}")
        print(f"    BLEU: {sample['bleu']:.3f}, ROUGE-1: {sample['rouge1']:.3f}")

# ====================== Model Loading Functions ======================
def load_sft_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading SFT checkpoint: {checkpoint_path}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    # Initialize model
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    if 'epoch' in checkpoint:
        print(f"Checkpoint info: Epoch {checkpoint['epoch']}, Loss {checkpoint.get('valid_loss', 'N/A')}")

    return model, tokenizer

def load_rl_model(model_dir, tokenizer_path, device='cuda'):
    """Load RL model from directory"""
    from transformers import AutoModelForCausalLM

    print(f"Loading RL model from directory: {model_dir}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load model from directory
    model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

    return model, tokenizer

def load_model_auto(model_path, tokenizer_path, device='cuda'):
    """
    Automatically detect and load model (SFT checkpoint or RL directory)

    Args:
        model_path: Path to model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        device: Device to load model on

    Returns:
        tuple: (model, tokenizer, model_type)
    """
    import os

    if os.path.isfile(model_path):
        # It's a file - assume SFT checkpoint
        print("Detected SFT checkpoint file")
        model, tokenizer = load_sft_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'sft'

    elif os.path.isdir(model_path):
        # It's a directory - assume RL model
        print("Detected RL model directory")
        model, tokenizer = load_rl_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'rl'

    else:
        raise ValueError(f"Model path {model_path} is neither a file nor a directory")

# ====================== Main Evaluation Function ======================
def run_evaluation(model_path, tokenizer_path, test_data_path, device='cuda'):
    """
    Run complete evaluation pipeline

    Args:
        model_path: Path to trained model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        test_data_path: Path to test JSON data
        device: Device to run evaluation on
    """

    print(f"Loading model from {model_path}...")
    model, tokenizer, model_type = load_model_auto(model_path, tokenizer_path, device)
    print(f"Loaded {model_type.upper()} model successfully")

    print(f"Loading evaluation data from {test_data_path}...")
    eval_dataset = TherapyEvaluationDataset(test_data_path, tokenizer_path)

    results = evaluate_model(model, tokenizer, eval_dataset, device=device)
    results['model_type'] = model_type

    print_evaluation_results(results)

    return results

# ====================== Usage Example ======================
if __name__ == "__main__":


    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4"  # Will auto-detect type
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/test.json"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Run evaluation (works for both SFT and RL)
    results = run_evaluation(MODEL_PATH, TOKENIZER_PATH, TEST_DATA_PATH, DEVICE)

    # Optionally save results
    import pickle
    with open('/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/Evaluation2/temp1dot3RL_evaluation_results.pkl', 'wb') as f:
      pickle.dump(results, f)

SFT No Emotion Hyperarameter Tuning

In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import itertools
import pickle
import os
from datetime import datetime
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set seed at import
set_seed(42)

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Model Output Extraction ======================
def extract_model_output_no_emotion(response):
    """Extract therapist text from model output - NO EMOTION VERSION"""
    response = response.strip()

    # Remove <eos> if present
    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Clean up any accidentally generated emotion tags (shouldn't happen)
    if '<therapist_emotion>' in response:
        response = response.split('<therapist_emotion>')[0].strip()
        print(f"WARNING: Model generated emotion tag (training issue)")

    # Remove any standalone emotion words at the end
    emotion_words = ['anger', 'sadness', 'depression', 'disgust', 'fear', 'joy', 'neutral']
    for emotion in emotion_words:
        pattern = r'\s+' + emotion + r'\s*$'
        response = re.sub(pattern, '', response, flags=re.IGNORECASE)

    return response

# ====================== Reference Data Processing ======================
class TherapyEvaluationDatasetNoEmotion:
    """Dataset class for evaluation data processing - NO EMOTION VERSION"""

    def __init__(self, json_path, tokenizer_path=None, subset_size=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self.subset_size = subset_size
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation - matches no-emotion training format"""
        print(f"Processing evaluation data (subset_size: {self.subset_size})...")

        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input prompt - MATCHES TRAINING FORMAT EXACTLY
                    input_parts = ["<bos>"]

                    if problem_type:
                        input_parts.extend(["<problem>", problem_type])

                    input_parts.extend(["<user>", combined_user_text])

                    if last_user_emotion:
                        input_parts.extend(["<user_emotion>", last_user_emotion])

                    input_parts.append("<therapist>")
                    input_text = "".join([self.tokenizer.bos_token] +
                                       ([f"<problem>{problem_type}"] if problem_type else []) +
                                       [f"<user>{combined_user_text}"] +
                                       ([f"<user_emotion>{last_user_emotion}"] if last_user_emotion else []) +
                                       ["<therapist>"])

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

                    # Early stopping if subset size is reached
                    if self.subset_size and len(self.evaluation_data) >= self.subset_size:
                        break

            # Early stopping if subset size is reached
            if self.subset_size and len(self.evaluation_data) >= self.subset_size:
                break

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetricsNoEmotion:
    """Class to compute evaluation metrics - NO EMOTION VERSION"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Fast Evaluation Function ======================
def evaluate_hyperparameters_no_emotion(model, tokenizer, evaluation_dataset, device='cuda',
                            max_new_tokens=64, top_p=0.8, top_k=0, temperature=1.0, do_sample=True,
                            verbose=False):
    """
    Fast evaluation for hyperparameter tuning - NO EMOTION VERSION

    Args:
        model: Trained model
        tokenizer: Model tokenizer
        evaluation_dataset: TherapyEvaluationDatasetNoEmotion instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        top_k: Top-k sampling parameter
        temperature: Temperature for sampling
        do_sample: Whether to use sampling
        verbose: Whether to print detailed progress

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetricsNoEmotion()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': []
    }

    model.eval()

    # Progress bar only if verbose
    if verbose:
        progress_bar = tqdm(evaluation_data, desc=f"Eval p={top_p:.2f} k={top_k} T={temperature:.1f}")
        data_iter = progress_bar
    else:
        data_iter = evaluation_data

    for i, sample in enumerate(data_iter):
        input_text = sample['input_text']
        reference_text = sample['reference_text']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text = extract_model_output_no_emotion(response)

        # Compute metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)

    if verbose and 'progress_bar' in locals():
        progress_bar.close()

    # Compute summary statistics
    summary = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'total_samples': len(evaluation_data),
        'top_p': top_p,
        'top_k': top_k,
        'temperature': temperature
    }

    return summary

# ====================== Model Loading Functions ======================
def load_no_emotion_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load no-emotion SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading no-emotion checkpoint: {checkpoint_path}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    # Initialize model
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    return model, tokenizer

# ====================== Hyperparameter Tuning Functions ======================
def compute_combined_score_no_emotion(results, weights=None):
    """
    Compute a combined score from multiple metrics - NO EMOTION VERSION

    Args:
        results: Dictionary containing metric results
        weights: Dictionary of weights for each metric

    Returns:
        Combined score (higher is better)
    """
    if weights is None:
        weights = {
            'avg_bleu': 0.25,
            'avg_rouge1': 0.20,
            'avg_rouge2': 0.20,
            'avg_rougeL': 0.20,
            'avg_meteor': 0.15
        }

    score = 0.0
    for metric, weight in weights.items():
        if metric in results:
            score += results[metric] * weight

    return score

def hyperparameter_search_no_emotion(model_path, tokenizer_path, test_data_path, device='cuda',
                         top_p_values=None, top_k_values=None, temperature_values=None, subset_size=100,
                         max_new_tokens=128, output_dir=None, weights=None):
    """
    Perform grid search over top_p, top_k, and temperature values - NO EMOTION VERSION

    Args:
        model_path: Path to trained model checkpoint
        tokenizer_path: Path to tokenizer
        test_data_path: Path to test JSON data
        device: Device to run evaluation on
        top_p_values: List of top_p values to test
        top_k_values: List of top_k values to test
        temperature_values: List of temperature values to test
        subset_size: Number of samples to use for evaluation
        max_new_tokens: Maximum tokens to generate
        output_dir: Directory to save results
        weights: Weights for combining metrics

    Returns:
        Dictionary with all results and best parameters
    """

    if top_p_values is None:
        top_p_values = [0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]

    if top_k_values is None:
        top_k_values = [0, 5, 10, 20, 40, 50]

    if temperature_values is None:
        temperature_values = [0.7, 0.8, 1.0, 1.1, 1.2]

    # Load model and tokenizer
    print(f"Loading no-emotion model from {model_path}...")
    model, tokenizer = load_no_emotion_model(model_path, tokenizer_path, device)
    print(f"Loaded no-emotion SFT model successfully")

    # Load evaluation dataset (subset)
    print(f"Loading evaluation data subset (size: {subset_size})...")
    eval_dataset = TherapyEvaluationDatasetNoEmotion(test_data_path, tokenizer_path, subset_size=subset_size)

    # Initialize results storage
    all_results = []
    best_score = -1
    best_params = None

    # Total combinations
    total_combinations = len(top_p_values) * len(top_k_values) * len(temperature_values)
    print(f"Testing {total_combinations} hyperparameter combinations...")
    print(f"top_p values: {top_p_values}")
    print(f"top_k values: {top_k_values}")
    print(f"temperature values: {temperature_values}")
    print(f"Subset size: {subset_size} samples")

    # Grid search
    combination_count = 0

    for top_p in top_p_values:
        for top_k in top_k_values:
            for temperature in temperature_values:
                combination_count += 1

                print(f"\n[{combination_count}/{total_combinations}] Testing top_p={top_p}, top_k={top_k}, temp={temperature}")

                # Evaluate with current hyperparameters
                results = evaluate_hyperparameters_no_emotion(
                    model, tokenizer, eval_dataset, device,
                    max_new_tokens=max_new_tokens,
                    top_p=top_p, top_k=top_k, temperature=temperature,
                    verbose=False
                )

                # Compute combined score
                combined_score = compute_combined_score_no_emotion(results, weights)
                results['combined_score'] = combined_score

                # Store results
                all_results.append(results)

                # Check if this is the best so far
                if combined_score > best_score:
                    best_score = combined_score
                    best_params = {'top_p': top_p, 'top_k': top_k, 'temperature': temperature}
                    print(f"  *** NEW BEST *** Score: {combined_score:.4f}")

                # Print current results
                print(f"  BLEU: {results['avg_bleu']:.4f}, "
                      f"ROUGE-1: {results['avg_rouge1']:.4f}, "
                      f"ROUGE-2: {results['avg_rouge2']:.4f}, "
                      f"ROUGE-L: {results['avg_rougeL']:.4f}, "
                      f"METEOR: {results['avg_meteor']:.4f}, "
                      f"Combined: {combined_score:.4f}")

    # Compile final results
    final_results = {
        'best_params': best_params,
        'best_score': best_score,
        'all_results': all_results,
        'model_path': model_path,
        'model_type': 'no_emotion_sft',
        'subset_size': subset_size,
        'search_space': {
            'top_p_values': top_p_values,
            'top_k_values': top_k_values,
            'temperature_values': temperature_values
        },
        'weights': weights or {
            'avg_bleu': 0.25,
            'avg_rouge1': 0.20,
            'avg_rouge2': 0.20,
            'avg_rougeL': 0.20,
            'avg_meteor': 0.15
        },
        'timestamp': datetime.now().isoformat()
    }

    # Print summary
    print(f"\n{'='*60}")
    print("NO-EMOTION HYPERPARAMETER SEARCH COMPLETED")
    print(f"{'='*60}")
    print(f"Best parameters: top_p={best_params['top_p']}, top_k={best_params['top_k']}, temperature={best_params['temperature']}")
    print(f"Best combined score: {best_score:.4f}")

    # Find best result details
    best_result = max(all_results, key=lambda x: x['combined_score'])
    print(f"\nBest result details:")
    print(f"  BLEU Score: {best_result['avg_bleu']:.4f}")
    print(f"  ROUGE-1: {best_result['avg_rouge1']:.4f}")
    print(f"  ROUGE-2: {best_result['avg_rouge2']:.4f}")
    print(f"  ROUGE-L: {best_result['avg_rougeL']:.4f}")
    print(f"  METEOR: {best_result['avg_meteor']:.4f}")
    print(f"  Combined Score: {best_result['combined_score']:.4f}")

    # Save results if output directory specified
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = os.path.join(output_dir, f"no_emotion_hyperparameter_search_{timestamp}.pkl")

        with open(results_file, 'wb') as f:
            pickle.dump(final_results, f)

        print(f"\nResults saved to: {results_file}")

        # Also save a summary CSV
        try:
            import pandas as pd
            df_results = pd.DataFrame(all_results)
            csv_file = os.path.join(output_dir, f"no_emotion_hyperparameter_results_{timestamp}.csv")
            df_results.to_csv(csv_file, index=False)
            print(f"CSV summary saved to: {csv_file}")
        except ImportError:
            print("pandas not available, skipping CSV export")

    return final_results

def analyze_no_emotion_results(results_file):
    """Analyze no-emotion hyperparameter search results"""
    with open(results_file, 'rb') as f:
        results = pickle.load(f)

    try:
        import pandas as pd
        # Convert to DataFrame
        df = pd.DataFrame(results['all_results'])

        print(f"No-Emotion Hyperparameter Search Analysis")
        print(f"=" * 50)
        print(f"Model: {results['model_path']}")
        print(f"Search completed: {results['timestamp']}")
        print(f"Total combinations tested: {len(df)}")

        # Top 5 results
        print(f"\nTop 5 Results:")
        top_5 = df.nlargest(5, 'combined_score')[['top_p', 'top_k', 'temperature', 'avg_bleu', 'avg_rouge1',
                                                  'avg_rouge2', 'avg_rougeL', 'avg_meteor', 'combined_score']]
        print(top_5.to_string(index=False))

        # Best for each metric
        print(f"\nBest parameters for each metric:")
        for metric in ['avg_bleu', 'avg_rouge1', 'avg_rouge2', 'avg_rougeL', 'avg_meteor']:
            best_row = df.loc[df[metric].idxmax()]
            print(f"  {metric}: top_p={best_row['top_p']}, top_k={best_row['top_k']}, temp={best_row['temperature']} (score: {best_row[metric]:.4f})")

        return df

    except ImportError:
        print("pandas not available, showing basic analysis")
        print(f"Best parameters: {results['best_params']}")
        print(f"Best score: {results['best_score']:.4f}")
        return results['all_results']

# ====================== Usage Example ======================
if __name__ == "__main__":
    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/therapy_checkpoints_4thFIXED_noemotions/best_model.ckpt"
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/train_processed_no_emotion_v1_tokenizer"
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/val.json"
    OUTPUT_DIR = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/hyperparameter_search"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    # Hyperparameter search space
    TOP_P_VALUES = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.00]
    TOP_K_VALUES = [0, 5, 10, 15, 20, 25, 30, 35]
    TEMPERATURE_VALUES = [1.0, 1.1, 1.2, 1.3]

    SUBSET_SIZE = 100

    CUSTOM_WEIGHTS = None

    results = hyperparameter_search_no_emotion(
        model_path=MODEL_PATH,
        tokenizer_path=TOKENIZER_PATH,
        test_data_path=TEST_DATA_PATH,
        device=DEVICE,
        top_p_values=TOP_P_VALUES,
        top_k_values=TOP_K_VALUES,
        temperature_values=TEMPERATURE_VALUES,
        subset_size=SUBSET_SIZE,
        max_new_tokens=128,
        output_dir=OUTPUT_DIR,
        weights=CUSTOM_WEIGHTS
    )

    print(f"\nBest hyperparameters found:")
    print(f"top_p: {results['best_params']['top_p']}")
    print(f"top_k: {results['best_params']['top_k']}")
    print(f"temperature: {results['best_params']['temperature']}")
    print(f"Score: {results['best_score']:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Loading no-emotion model from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/therapy_checkpoints_4thFIXED_noemotions/best_model.ckpt...
Loading no-emotion checkpoint: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/therapy_checkpoints_4thFIXED_noemotions/best_model.ckpt


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Loaded no-emotion SFT model successfully
Loading evaluation data subset (size: 100)...
Processing evaluation data (subset_size: 100)...


Processing evaluation data:  25%|██▌       | 26/102 [00:00<00:00, 4784.03it/s]

Processed 100 evaluation samples
Testing 288 hyperparameter combinations...
top_p values: [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
top_k values: [0, 5, 10, 15, 20, 25, 30, 35]
temperature values: [1.0, 1.1, 1.2, 1.3]
Subset size: 100 samples

[1/288] Testing top_p=0.6, top_k=0, temp=1.0


  *** NEW BEST *** Score: 0.0808
  BLEU: 0.0196, ROUGE-1: 0.1454, ROUGE-2: 0.0424, ROUGE-L: 0.1413, METEOR: 0.0672, Combined: 0.0808

[2/288] Testing top_p=0.6, top_k=0, temp=1.1
  BLEU: 0.0086, ROUGE-1: 0.1131, ROUGE-2: 0.0089, ROUGE-L: 0.1061, METEOR: 0.0428, Combined: 0.0542

[3/288] Testing top_p=0.6, top_k=0, temp=1.2
  BLEU: 0.0092, ROUGE-1: 0.0839, ROUGE-2: 0.0057, ROUGE-L: 0.0775, METEOR: 0.0366, Combined: 0.0412

[4/288] Testing top_p=0.6, top_k=0, temp=1.3
  BLEU: 0.0038, ROUGE-1: 0.0594, ROUGE-2: 0.0049, ROUGE-L: 0.0575, METEOR: 0.0276, Combined: 0.0294

[5/288] Testing top_p=0.6, top_k=5, temp=1.0
  BLEU: 0.0162, ROUGE-1: 0.1313, ROUGE-2: 0.0259, ROUGE-L: 0.1230, METEOR: 0.0496, Combined: 0.0675

[6/288] Testing top_p=0.6, top_k=5, temp=1.1
  BLEU: 0.0158, ROUGE-1: 0.1405, ROUGE-2: 0.0401, ROUGE-L: 0.1339, METEOR: 0.0594, Combined: 0.0758

[7/288] Testing top_p=0.6, top_k=5, temp=1.2
  BLEU: 0.0075, ROUGE-1: 0.1040, ROUGE-2: 0.0098, ROUGE-L: 0.0993, METEOR: 0.0352, Combined

SFT No Emotion EValuation

In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AutoTokenizer, AutoModelForSequenceClassification
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns from SFT data"""
    expression_pattern = r'\s*(The (?:speaker|emotion state)[^.]*\.(?:[^.]*\.)*)'
    match = re.search(expression_pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return text[:match.start()].strip()
    return text.strip()

# ====================== Model Loading Functions ======================
def load_no_emotion_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load no-emotion trained model"""
    print(f"Loading no-emotion model: {checkpoint_path}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    # Initialize model
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    print(f"No-emotion model loaded with vocab size: {len(tokenizer)}")
    return model, tokenizer

def load_baseline_gpt2(device='cuda'):
    """Load vanilla GPT-2 model"""
    print("Loading vanilla GPT-2 model...")
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.to(device)
    model.eval()
    print(f"Vanilla GPT-2 loaded with vocab size: {len(tokenizer)}")
    return model, tokenizer

def load_emotion_classifier(device='cuda'):
    """Load RoBERTa emotion classifier"""
    print("Loading emotion classifier...")
    emotion_tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
    emotion_model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")
    emotion_model.to(device)
    emotion_model.eval()
    return emotion_model, emotion_tokenizer

def predict_emotion_roberta(text, emotion_model, emotion_tokenizer, device='cuda'):
    """Predict emotion using RoBERTa model, constrained to therapy emotions"""
    if not text.strip():
        return "neutral", 0.0

    therapy_emotions = ['anger', 'joy', 'neutral', 'sadness', 'depression', 'disgust', 'fear']

    # Tokenize and predict
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = emotion_model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Get all emotion labels from RoBERTa model
    emotion_labels = emotion_model.config.id2label

    # Enhanced mapping based on the 28 RoBERTa emotions
    emotion_mapping = {
        # Direct matches
        'anger': 'anger',
        'joy': 'joy',
        'sadness': 'sadness',
        'fear': 'fear',
        'disgust': 'disgust',
        'neutral': 'neutral',

        # Joy-related emotions
        'amusement': 'joy',
        'excitement': 'joy',
        'gratitude': 'joy',
        'love': 'joy',
        'optimism': 'joy',
        'pride': 'joy',
        'relief': 'joy',
        'admiration': 'joy',

        # Sadness-related emotions (potential depression indicators)
        'disappointment': 'sadness',
        'embarrassment': 'sadness',
        'grief': 'sadness',
        'remorse': 'sadness',

        # Anger-related emotions
        'annoyance': 'anger',
        'disapproval': 'anger',

        # Fear-related emotions
        'nervousness': 'fear',

        # Neutral-related emotions
        'approval': 'neutral',
        'caring': 'neutral',
        'confusion': 'neutral',
        'curiosity': 'neutral',
        'desire': 'neutral',
        'realization': 'neutral',
        'surprise': 'neutral'
    }

    therapy_scores = {emotion: 0.0 for emotion in therapy_emotions}

    for class_id, score in enumerate(predictions[0]):
        go_emotion = emotion_labels[class_id]
        therapy_emotion = emotion_mapping.get(go_emotion, 'neutral')
        therapy_scores[therapy_emotion] += score.item()

    best_emotion = max(therapy_scores, key=therapy_scores.get)
    best_score = therapy_scores[best_emotion]

    if best_emotion == 'sadness':
        depression_indicators = ['grief', 'remorse', 'disappointment', 'embarrassment']
        depression_score = sum(predictions[0][class_id].item()
                              for class_id, label in emotion_labels.items()
                              if label in depression_indicators)

        if depression_score > 0.2 or (emotion_labels.get(predictions[0].argmax().item()) in ['grief', 'remorse']):
            best_emotion = 'depression'
            best_score = depression_score

    return best_emotion, best_score

# ====================== Reference Data Processing ======================
class TherapyEvaluationDatasetNoEmotion:
    """Dataset class for evaluation data processing - No Emotion Version"""

    def __init__(self, json_path):
        with open(json_path, 'r', encoding='utf-8') as f:
            self._data = json.load(f)

        self.evaluation_data = []
        self._process_evaluation_data()

    def _process_evaluation_data(self):
        """Process dialog data for evaluation"""
        for conv in tqdm(self._data, desc="Processing evaluation data"):
            problem_type = conv.get("problem_type", "").strip()
            dialog = conv.get("dialog", [])

            user_text_parts = []
            user_emotions = []

            for turn in dialog:
                speaker = turn.get("speaker", "")
                text = clean_therapy_text(turn.get("text", ""))
                emotion = turn.get("emotion", "").strip()

                if speaker != "sys":  # User turn
                    if text:
                        user_text_parts.append(text)
                    if emotion:
                        user_emotions.append(emotion)
                else:  # Therapist turn
                    if not user_text_parts:
                        continue

                    therapist_text = clean_therapy_text(text)
                    therapist_emotion = emotion

                    combined_user_text = " ".join(user_text_parts)
                    last_user_emotion = user_emotions[-1] if user_emotions else ""

                    # Create input in no-emotion training format
                    input_parts = ["<bos>"]
                    if problem_type:
                        input_parts.extend(["<problem>", problem_type])
                    input_parts.extend(["<user>", combined_user_text])
                    if last_user_emotion:
                        input_parts.extend(["<user_emotion>", last_user_emotion])
                    input_parts.append("<therapist>")

                    input_text = "".join(input_parts)

                    self.evaluation_data.append({
                        'input_text': input_text,
                        'reference_text': therapist_text,
                        'reference_emotion': therapist_emotion,
                        'user_input': combined_user_text,
                        'user_emotion': last_user_emotion,
                        'problem_type': problem_type
                    })

                    user_text_parts = []
                    user_emotions = []

        print(f"Processed {len(self.evaluation_data)} evaluation samples")

    def get_evaluation_data(self):
        return self.evaluation_data

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Model Evaluation ======================
def evaluate_model_with_emotion_detection(model, tokenizer, evaluation_dataset, device='cuda',
                                         top_p=0.6, top_k=0, temperature=1.0, max_new_tokens=64, model_name="Model"):
    """
    Evaluate model with RoBERTa emotion detection
    """
    # Load emotion classifier
    emotion_model, emotion_tokenizer = load_emotion_classifier(device)
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'sample_outputs': []
    }

    emotion_confusion = defaultdict(lambda: defaultdict(int))
    model.eval()

    print(f"Evaluating {model_name} on {len(evaluation_data)} samples...")

    # Create progress bar
    progress_bar = tqdm(evaluation_data, desc=f"Evaluating {model_name}")

    for i, sample in enumerate(progress_bar):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        if "baseline" in model_name.lower():
            user_match = re.search(r'<user>(.*?)(?:<user_emotion>|<therapist>)', input_text)
            if user_match:
                processed_input = user_match.group(1).strip()
            else:
                processed_input = sample['user_input']
        else:
            # For trained model, use full structured input
            processed_input = input_text

        # Tokenize input
        input_ids = tokenizer.encode(processed_input, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        predicted_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

        # Predict emotion using RoBERTa
        predicted_emotion, emotion_confidence = predict_emotion_roberta(
            predicted_text, emotion_model, emotion_tokenizer, device
        )

        # Compute text generation metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower())

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)

        emotion_confusion[reference_emotion.lower()][predicted_emotion.lower()] += 1

        # Store sample outputs for inspection
        if i < 5:
            results['sample_outputs'].append({
                'input': input_text,
                'processed_input': processed_input,
                'reference_text': reference_text,
                'predicted_text': predicted_text,
                'reference_emotion': reference_emotion,
                'predicted_emotion': predicted_emotion,
                'emotion_confidence': emotion_confidence,
                'bleu': bleu,
                'rouge1': rouge['rouge1'],
                'meteor': meteor
            })

        # Update progress
        if (i + 1) % 20 == 0:
            avg_bleu = np.mean(results['bleu_scores'])
            avg_emotion_acc = np.mean(results['emotion_accuracy'])
            progress_bar.set_description(f"Evaluating {model_name} - BLEU: {avg_bleu:.3f}, Emotion: {avg_emotion_acc:.3f}")

    progress_bar.close()

    results['summary'] = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'total_samples': len(evaluation_data)
    }

    results['emotion_confusion_matrix'] = dict(emotion_confusion)
    results['model_name'] = model_name
    results['generation_params'] = {
        'top_p': top_p,
        'top_k': top_k,
        'temperature': temperature,
        'max_new_tokens': max_new_tokens
    }

    return results

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    summary = results['summary']
    model_name = results.get('model_name', 'Unknown')

    print("\n" + "="*60)
    print(f"{model_name.upper()} EVALUATION RESULTS")
    print("="*60)

    # Show generation parameters
    gen_params = results.get('generation_params', {})
    if gen_params:
        print(f"Generation Params: top_p={gen_params.get('top_p', 'N/A')}, "
              f"max_tokens={gen_params.get('max_new_tokens', 'N/A')}")

    print(f"\nText Generation Metrics:")
    print(f"  BLEU Score:     {summary['avg_bleu']:.4f}")
    print(f"  ROUGE-1:        {summary['avg_rouge1']:.4f}")
    print(f"  ROUGE-2:        {summary['avg_rouge2']:.4f}")
    print(f"  ROUGE-L:        {summary['avg_rougeL']:.4f}")
    print(f"  METEOR:         {summary['avg_meteor']:.4f}")

    print(f"\nEmotion Prediction (via RoBERTa):")
    print(f"  Emotion Accuracy:    {summary['emotion_accuracy']:.4f}")

    print(f"\nDataset Info:")
    print(f"  Total Samples:  {summary['total_samples']}")

    print(f"\nSample Outputs:")
    for i, sample in enumerate(results['sample_outputs'][:3], 1):
        print(f"\n  Sample {i}:")
        print(f"    Input: {sample['input'][:80]}...")
        print(f"    Processed: {sample['processed_input'][:60]}...")
        print(f"    Reference: {sample['reference_text'][:60]}...")
        print(f"    Predicted: {sample['predicted_text'][:60]}...")
        print(f"    Ref Emotion: {sample['reference_emotion']}")
        print(f"    Pred Emotion: {sample['predicted_emotion']} (conf: {sample['emotion_confidence']:.3f})")
        print(f"    BLEU: {sample['bleu']:.3f}, ROUGE-1: {sample['rouge1']:.3f}")

def compare_models(results_list):
    """Compare multiple model results"""
    print("\n" + "="*80)
    print("MODEL COMPARISON")
    print("="*80)

    print(f"{'Model':<30} {'BLEU':<8} {'ROUGE-1':<8} {'ROUGE-L':<8} {'METEOR':<8} {'Emotion Acc':<12}")
    print("-" * 80)

    for results in results_list:
        summary = results['summary']
        model_name = results.get('model_name', 'Unknown')
        print(f"{model_name:<30} {summary['avg_bleu']:<8.4f} {summary['avg_rouge1']:<8.4f} "
              f"{summary['avg_rougeL']:<8.4f} {summary['avg_meteor']:<8.4f} {summary['emotion_accuracy']:<12.4f}")

# ====================== Usage Example ======================
if __name__ == "__main__":
    NO_EMOTION_MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/therapy_checkpoints_4thFIXED_noemotions/best_model.ckpt"
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/train_processed_no_emotion_v1_tokenizer"
    TEST_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/test.json"
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    eval_dataset = TherapyEvaluationDatasetNoEmotion(TEST_DATA_PATH)

    results_list = []

    print("=== EVALUATING NO-EMOTION TRAINED MODEL ===")
    no_emotion_model, no_emotion_tokenizer = load_no_emotion_model(NO_EMOTION_MODEL_PATH, TOKENIZER_PATH, DEVICE)
    no_emotion_results = evaluate_model_with_emotion_detection(
        no_emotion_model, no_emotion_tokenizer, eval_dataset, DEVICE,
        top_p=0.6, top_k=0, temperature=1.0, max_new_tokens=128, model_name="No-Emotion Trained Model"
    )
    print_evaluation_results(no_emotion_results)
    results_list.append(no_emotion_results)


    compare_models(results_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
Processing evaluation data: 100%|██████████| 102/102 [00:00<00:00, 7074.66it/s]


Processed 454 evaluation samples
=== EVALUATING NO-EMOTION TRAINED MODEL ===
Loading no-emotion model: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/SFTNoemotions/therapy_checkpoints_4thFIXED_noemotions/best_model.ckpt


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Evaluating No-Emotion Trained Model on 454 samples...


Evaluating No-Emotion Trained Model - BLEU: 0.011, Emotion: 0.807: 100%|██████████| 454/454 [00:47<00:00,  9.56it/s]


NO-EMOTION TRAINED MODEL EVALUATION RESULTS
Generation Params: top_p=0.6, max_tokens=128

Text Generation Metrics:
  BLEU Score:     0.0106
  ROUGE-1:        0.1160
  ROUGE-2:        0.0159
  ROUGE-L:        0.1058
  METEOR:         0.0433

Emotion Prediction (via RoBERTa):
  Emotion Accuracy:    0.8084

Dataset Info:
  Total Samples:  454

Sample Outputs:

  Sample 1:
    Input: <bos><problem>Breakups or Divorce<user>I mean, shit, don't you know that men are...
    Processed: <bos><problem>Breakups or Divorce<user>I mean, shit, don't y...
    Reference: But I thought you said...
    Predicted: And what do you think it was about you that he lost touch wi...
    Ref Emotion: neutral
    Pred Emotion: neutral (conf: 0.984)
    BLEU: 0.014, ROUGE-1: 0.105

  Sample 2:
    Input: <bos><problem>Breakups or Divorce<user>Well, we were watching tv.But before that...
    Processed: <bos><problem>Breakups or Divorce<user>Well, we were watchin...
    Reference: So you were the one who came up wi

**ESCONV EVALUATION**

RL

In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns and clean text"""
    if not text:
        return ""
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# ====================== ESConv Dataset Processing ======================
class ESConvEvaluationDataset:
    """Dataset class for ESConv evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, dict):
                self._data = [data]
            else:
                self._data = data

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            # Use default tokenizer if path not provided
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self._process_esconv_data()

    def _process_esconv_data(self):
        """Process ESConv dialog data for evaluation"""
        print("Processing ESConv data...")

        for conv_idx, conv in enumerate(tqdm(self._data, desc="Processing conversations")):
            # Extract conversation metadata
            problem_type = conv.get("problem_type", "").strip()
            emotion_type = conv.get("emotion_type", "").strip()
            dialog = conv.get("dialog", [])

            if not dialog:
                continue

            if conv_idx < 3:
                print(f"\nProcessing conversation {conv_idx + 1}:")
                print(f"  Problem type: {problem_type}")
                print(f"  Emotion type: {emotion_type}")
                print(f"  Dialog length: {len(dialog)} turns")

            # Track conversation state
            user_messages = []  # Accumulate user messages

            for turn_idx, turn in enumerate(dialog):
                speaker = turn.get("speaker", "").strip()
                content = clean_therapy_text(turn.get("content", ""))

                if not content:
                    continue

                if speaker == "seeker":
                    # Accumulate user (seeker) messages
                    user_messages.append(content)

                elif speaker == "supporter":
                    if user_messages:  # Only if we have accumulated user messages

                        # Combine all user messages for this interaction
                        combined_user_text = " ".join(user_messages)
                        therapist_response = content

                        # Create structured input in the expected format
                        input_parts = []
                        if problem_type:
                            input_parts.append(f"<problem>{problem_type}")

                        input_parts.append(f"<user>{combined_user_text}")

                        if emotion_type:
                            input_parts.append(f"<user_emotion>{emotion_type}")

                        input_parts.append("<therapist>")
                        input_text = " ".join(input_parts)

                        # Create evaluation sample
                        eval_sample = {
                            'input_text': input_text,
                            'reference_text': therapist_response,
                            'reference_emotion': emotion_type,
                            'user_input': combined_user_text,
                            'user_emotion': emotion_type,
                            'problem_type': problem_type,
                            'conversation_id': conv_idx,
                            'turn_id': turn_idx
                        }

                        self.evaluation_data.append(eval_sample)

                        if len(self.evaluation_data) <= 3:
                            print(f"\n  Sample {len(self.evaluation_data)}:")
                            print(f"    Input: {input_text[:100]}...")
                            print(f"    Reference: {therapist_response[:100]}...")

                        # Reset user messages for next interaction
                        user_messages = []

        print(f"\nProcessed {len(self.evaluation_data)} evaluation samples from {len(self._data)} conversations")

    def get_evaluation_data(self):
        return self.evaluation_data

    def print_sample_formats(self, num_samples=3):
        """Print sample input/output formats for verification"""
        print(f"\n{'='*80}")
        print("ESCONV SAMPLE INPUT/OUTPUT FORMATS")
        print('='*80)

        for i, sample in enumerate(self.evaluation_data[:num_samples]):
            print(f"\nSample {i+1}:")
            print(f"  Problem Type: {sample['problem_type']}")
            print(f"  User Emotion: {sample['user_emotion']}")
            print(f"  Input Format: {sample['input_text'][:150]}...")
            print(f"  Reference Response: {sample['reference_text'][:100]}...")
            print(f"  User Input Only: {sample['user_input'][:100]}...")

# ====================== Model Format Detection ======================
def detect_model_format(model, tokenizer, device='cuda', test_inputs=None):
    """
    Test model output format (should be consistent since both SFT and RL use same format)

    Args:
        model: The model to test
        tokenizer: Model tokenizer
        device: Device to run test on
        test_inputs: List of test input strings, uses defaults if None

    Returns:
        str: 'standard' if using <therapist_emotion> format, 'unknown' otherwise
    """
    if test_inputs is None:
        test_inputs = [
            "<problem>anxiety <user>I'm worried about work <user_emotion>anxiety <therapist>",
            "<problem>depression <user>I feel very sad <user_emotion>sadness <therapist>",
            "<problem>relationship <user>My partner doesn't understand me <user_emotion>anger <therapist>"
        ]

    emotion_tag_count = 0
    total_tests = len(test_inputs)

    model.eval()
    with torch.no_grad():
        for test_input in test_inputs:
            # Tokenize and generate
            input_ids = tokenizer.encode(test_input, return_tensors='pt').to(device)
            outputs = model.generate(
                input_ids,
                max_new_tokens=128,
                do_sample=True,
                top_p=0.8,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

            # Decode response
            new_tokens = outputs[0][len(input_ids[0]):]
            response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

            # Check for emotion tag
            if '<therapist_emotion>' in response:
                emotion_tag_count += 1

    # Determine format
    if emotion_tag_count >= total_tests * 0.5:
        return 'standard'
    else:
        return 'unknown'

# ====================== Model Output Extraction ======================
def extract_model_output(response):
    """
    Extract therapist text and emotion from model output.
    Both SFT and RL models use format: 'text <therapist_emotion> emotion<eos>'

    Returns:
        therapist_text (str): Text before <therapist_emotion>
        emotion (str): Emotion word after <therapist_emotion>
        has_emotion_tag (bool): Whether emotion tag was found
    """
    response = response.strip()

    # Remove <eos> if present
    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Find <therapist_emotion> tag
    emotion_pattern = r'<therapist_emotion>'
    emotion_match = re.search(emotion_pattern, response)

    if emotion_match:
        # Extract text before <therapist_emotion>
        therapist_text = response[:emotion_match.start()].strip()

        # Extract emotion part after <therapist_emotion>
        emotion_part = response[emotion_match.end():].strip()

        # Get the first word as emotion
        emotion_words = emotion_part.split()
        emotion = emotion_words[0].lower() if emotion_words else ""

        return therapist_text, emotion, True
    else:
        # No emotion tag found, return entire response as text
        return response, "", False

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Model Evaluation ======================
def evaluate_model_esconv(model, tokenizer, evaluation_dataset, device='cuda', max_new_tokens=128,
                         top_p=0.7, top_k=10, temperature=1.2, do_sample=True):
    """
    Evaluate model on ESConv therapy dataset

    Args:
        model: Trained model (SFT or RL)
        tokenizer: Model tokenizer
        evaluation_dataset: ESConvEvaluationDataset instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        do_sample: Whether to use sampling

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
        'sample_outputs': []
    }

    emotion_confusion = defaultdict(lambda: defaultdict(int))

    model.eval()

    # Detect model output format
    detected_format = detect_model_format(model, tokenizer, device)
    print(f"Output format check: {detected_format.upper()}")

    print(f"Evaluating model on {len(evaluation_data)} ESConv samples...")

    # Create progress bar with custom format
    progress_bar = tqdm(evaluation_data, desc="Evaluating ESConv",
                       bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')

    for i, sample in enumerate(progress_bar):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text, predicted_emotion, has_emotion = extract_model_output(response)

        # Compute text generation metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower()) if has_emotion else False

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

        # Update confusion matrix
        if has_emotion:
            emotion_confusion[reference_emotion.lower()][predicted_emotion.lower()] += 1
        else:
            emotion_confusion[reference_emotion.lower()]['no_prediction'] += 1

        # Store sample outputs for inspection
        if i < 10:  # Store first 10 samples
            results['sample_outputs'].append({
                'input': input_text,
                'reference_text': reference_text,
                'predicted_text': predicted_text,
                'reference_emotion': reference_emotion,
                'predicted_emotion': predicted_emotion,
                'full_response': response,
                'bleu': bleu,
                'rouge1': rouge['rouge1'],
                'meteor': meteor
            })

        # Show live samples during evaluation (every 20 samples)
        if (i + 1) % 20 == 0 or i < 5:
            # Update progress bar description with current sample info
            sample_info = f"Sample {i+1}: BLEU={bleu:.3f}, Emotion={'✓' if emotion_correct else '✗'}"
            progress_bar.set_description(f"Evaluating ESConv - {sample_info}")

            if (i + 1) % 50 == 0 or i < 3:
                print(f"\n" + "="*80)
                print(f"ESCONV LIVE SAMPLE {i+1}/{len(evaluation_data)}")
                print("="*80)
                print(f"Input: {input_text[:120]}...")
                print(f"Reference: {reference_text[:100]}...")
                print(f"Generated: {predicted_text[:100]}...")
                print(f"Full Response: {response[:150]}...")
                print(f"Ref Emotion: {reference_emotion} | Pred Emotion: {predicted_emotion} | Match: {'✓' if emotion_correct else '✗'}")
                print(f"BLEU: {bleu:.4f} | ROUGE-1: {rouge['rouge1']:.4f} | METEOR: {meteor:.4f}")
                print("="*80)
        else:
            # Just update progress bar with running averages
            if i > 0:
                avg_bleu = np.mean(results['bleu_scores'])
                avg_emotion_acc = np.mean(results['emotion_accuracy'])
                progress_bar.set_description(f"Evaluating ESConv - Avg BLEU: {avg_bleu:.3f}, Emotion Acc: {avg_emotion_acc:.3f}")

    progress_bar.close()

    # Compute summary statistics
    results['summary'] = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data)
    }

    results['emotion_confusion_matrix'] = dict(emotion_confusion)
    results['detected_format'] = detected_format

    # Print final progress summary
    print(f"\n{'='*60}")
    print(f"ESCONV EVALUATION COMPLETED!")
    print(f"{'='*60}")
    print(f"Processed {len(evaluation_data)} samples")
    print(f"Average BLEU: {results['summary']['avg_bleu']:.4f}")
    print(f"Average ROUGE-1: {results['summary']['avg_rouge1']:.4f}")
    print(f"Emotion Accuracy: {results['summary']['emotion_accuracy']:.4f}")
    print(f"Emotion Tag Coverage: {results['summary']['emotion_tag_coverage']:.4f}")
    print(f"{'='*60}")

    return results

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    summary = results['summary']

    print("\n" + "="*60)
    print("ESCONV THERAPY MODEL EVALUATION RESULTS")
    print("="*60)
    print(f"Model Type: {results.get('model_type', 'Unknown').upper()}")
    print(f"Output Format: {results.get('detected_format', 'Unknown').upper()}")

    print(f"\nText Generation Metrics:")
    print(f"  BLEU Score:     {summary['avg_bleu']:.4f}")
    print(f"  ROUGE-1:        {summary['avg_rouge1']:.4f}")
    print(f"  ROUGE-2:        {summary['avg_rouge2']:.4f}")
    print(f"  ROUGE-L:        {summary['avg_rougeL']:.4f}")
    print(f"  METEOR:         {summary['avg_meteor']:.4f}")

    print(f"\nEmotion Prediction:")
    print(f"  Emotion Accuracy:    {summary['emotion_accuracy']:.4f}")
    print(f"  Emotion Tag Coverage: {summary['emotion_tag_coverage']:.4f}")

    print(f"\nDataset Info:")
    print(f"  Total Samples:  {summary['total_samples']}")

    # Print emotion confusion matrix
    if 'emotion_confusion_matrix' in results:
        print(f"\nEmotion Confusion Matrix:")
        confusion = results['emotion_confusion_matrix']
        for ref_emotion, pred_dict in confusion.items():
            print(f"  {ref_emotion}:")
            for pred_emotion, count in pred_dict.items():
                print(f"    -> {pred_emotion}: {count}")

    print(f"\nSample Outputs:")
    for i, sample in enumerate(results['sample_outputs'][:3], 1):
        print(f"\n  Sample {i}:")
        print(f"    Input: {sample['input'][:100]}...")
        print(f"    Reference: {sample['reference_text'][:80]}...")
        print(f"    Predicted: {sample['predicted_text'][:80]}...")
        print(f"    Ref Emotion: {sample['reference_emotion']}")
        print(f"    Pred Emotion: {sample['predicted_emotion']}")
        print(f"    BLEU: {sample['bleu']:.3f}, ROUGE-1: {sample['rouge1']:.3f}")

# ====================== Model Loading Functions ======================
def load_sft_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading SFT checkpoint: {checkpoint_path}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    if 'epoch' in checkpoint:
        print(f"Checkpoint info: Epoch {checkpoint['epoch']}, Loss {checkpoint.get('valid_loss', 'N/A')}")

    return model, tokenizer

def load_rl_model(model_dir, tokenizer_path, device='cuda'):
    """Load RL model from directory"""
    from transformers import AutoModelForCausalLM

    print(f"Loading RL model from directory: {model_dir}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

    return model, tokenizer

def load_model_auto(model_path, tokenizer_path, device='cuda'):
    """
    Automatically detect and load model (SFT checkpoint or RL directory)

    Args:
        model_path: Path to model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        device: Device to load model on

    Returns:
        tuple: (model, tokenizer, model_type)
    """
    import os

    if os.path.isfile(model_path):
        print("Detected SFT checkpoint file")
        model, tokenizer = load_sft_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'sft'

    elif os.path.isdir(model_path):
        # It's a directory - assume RL model
        print("Detected RL model directory")
        model, tokenizer = load_rl_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'rl'

    else:
        raise ValueError(f"Model path {model_path} is neither a file nor a directory")

# ====================== Main Evaluation Function ======================
def run_esconv_evaluation(model_path, tokenizer_path, esconv_data_path, device='cuda'):
    """
    Run complete evaluation pipeline on ESConv data

    Args:
        model_path: Path to trained model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        esconv_data_path: Path to ESConv JSON data
        device: Device to run evaluation on
    """

    # Load model and tokenizer (auto-detect type)
    print(f"Loading model from {model_path}...")
    model, tokenizer, model_type = load_model_auto(model_path, tokenizer_path, device)
    print(f"Loaded {model_type.upper()} model successfully")

    print(f"Loading ESConv evaluation data from {esconv_data_path}...")
    eval_dataset = ESConvEvaluationDataset(esconv_data_path, tokenizer_path)

    # Print sample formats to verify conversion
    eval_dataset.print_sample_formats(3)

    # Run evaluation
    results = evaluate_model_esconv(model, tokenizer, eval_dataset, device=device)
    results['model_type'] = model_type
    results['dataset_type'] = 'esconv'

    # Print results
    print_evaluation_results(results)

    return results

# ====================== Usage Example ======================
if __name__ == "__main__":


    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4"

    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"
    ESCONV_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/PESConv.json"  # Update this path to your ESConv data
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    results = run_esconv_evaluation(MODEL_PATH, TOKENIZER_PATH, ESCONV_DATA_PATH, DEVICE)

    import pickle
    with open('/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/Evaluation2/ESConv_RL_evaluation_results2.pkl', 'wb') as f:
        pickle.dump(results, f)

    print("\nEvaluation completed! Results saved to ESConv_RL_evaluation_results2.pkl")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading model from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4...
Detected RL model directory
Loading RL model from directory: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4


Some weights of the model checkpoint at /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/8m19enhanced_ppo_model/epoch_4 were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded RL model successfully
Loading ESConv evaluation data from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/PESConv.json...
Processing ESConv data...


Processing conversations:  28%|██▊       | 365/1300 [00:00<00:00, 3649.62it/s]

...


ESCONV LIVE SAMPLE 14500/14642
Input: <problem>breakup with partner <user>Yeah she was a little violent. I was honestly a little scared for my life for a seco...
Reference: Have you talked since then?...
Generated: Did she say she was a little scared for your life?...
Full Response: Did she say she was a little scared for your life? <therapist_emotion> neutral<eos>...
Ref Emotion: sadness | Pred Emotion: neutral | Match: ✗
BLEU: 0.0000 | ROUGE-1: 0.0000 | METEOR: 0.0000


Evaluating ESConv - Avg BLEU: 0.004, Emotion Acc: 0.000: 100%|█████████▉| 14601/14642 [28:49<00:04, 10.07it/s]


ESCONV LIVE SAMPLE 14600/14642
Input: <problem>job crisis <user>Yes It is frustrating. When I think about the financial issues I feel depressed <user_emotion>...
Reference: I compleyely understand I have also lost my career due to covid. Dont give up, staying positive and ...
Generated: You're depressed?...
Full Response: You're depressed? <therapist_emotion> neutral<eos>...
Ref Emotion: depression | Pred Emotion: neutral | Match: ✗
BLEU: 0.0000 | ROUGE-1: 0.0645 | METEOR: 0.0000


Evaluating ESConv - Avg BLEU: 0.004, Emotion Acc: 0.000: 100%|██████████| 14642/14642 [28:53<00:00,  8.45it/s]


ESCONV EVALUATION COMPLETED!
Processed 14642 samples
Average BLEU: 0.0042
Average ROUGE-1: 0.1110
Emotion Accuracy: 0.0000
Emotion Tag Coverage: 0.9992

ESCONV THERAPY MODEL EVALUATION RESULTS
Model Type: RL
Output Format: STANDARD

Text Generation Metrics:
  BLEU Score:     0.0042
  ROUGE-1:        0.1110
  ROUGE-2:        0.0092
  ROUGE-L:        0.1002
  METEOR:         0.0290

Emotion Prediction:
  Emotion Accuracy:    0.0000
  Emotion Tag Coverage: 0.9992

Dataset Info:
  Total Samples:  14642

Emotion Confusion Matrix:
  anxiety:
    -> neutral: 3907
    -> emotional: 2
    -> you're: 14
    -> of: 1
    -> anger?: 1
    -> no_prediction: 4
    -> parent?: 1
    -> you: 1
    -> if: 1
    -> anxiety.: 1
  anger:
    -> neutral: 1255
    -> you're: 1
    -> anger?: 1
  fear:
    -> neutral: 1048
    -> <therapist_emotion>: 1
  depression:
    -> neutral: 3816
    -> no_prediction: 4
    -> cause: 2
    -> s?: 1
    -> <therapist_emotion>: 1
    -> anger.: 1
    -> will: 1
    -> 

SFT

In [ ]:
import json
import re
import torch
from transformers import GPT2Tokenizer
from datasets import Dataset as HFDataset
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import nltk
from tqdm import tqdm
import numpy as np
from collections import defaultdict

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

# ====================== Text Cleaning Functions ======================
def clean_therapy_text(text):
    """Remove descriptive text patterns and clean text"""
    if not text:
        return ""
    # Remove newlines and extra whitespace
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# ====================== ESConv Dataset Processing ======================
class ESConvEvaluationDataset:
    """Dataset class for ESConv evaluation data processing"""

    def __init__(self, json_path, tokenizer_path=None):
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            if isinstance(data, dict):
                self._data = [data]
            else:
                self._data = data

        if tokenizer_path:
            self.tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
        else:
            # Use default tokenizer if path not provided
            from transformers import AutoTokenizer
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2')

        self.evaluation_data = []
        self._process_esconv_data()

    def _process_esconv_data(self):
        """Process ESConv dialog data for evaluation"""
        print("Processing ESConv data...")

        for conv_idx, conv in enumerate(tqdm(self._data, desc="Processing conversations")):
            problem_type = conv.get("problem_type", "").strip()
            emotion_type = conv.get("emotion_type", "").strip()
            dialog = conv.get("dialog", [])

            if not dialog:
                continue

            if conv_idx < 3:
                print(f"\nProcessing conversation {conv_idx + 1}:")
                print(f"  Problem type: {problem_type}")
                print(f"  Emotion type: {emotion_type}")
                print(f"  Dialog length: {len(dialog)} turns")

            user_messages = []  # Accumulate user messages

            for turn_idx, turn in enumerate(dialog):
                speaker = turn.get("speaker", "").strip()
                content = clean_therapy_text(turn.get("content", ""))

                if not content:
                    continue

                if speaker == "seeker":
                    # Accumulate user (seeker) messages
                    user_messages.append(content)

                elif speaker == "supporter":
                    if user_messages:

                        # Combine all user messages for this interaction
                        combined_user_text = " ".join(user_messages)
                        therapist_response = content

                        # Create structured input in the expected format
                        input_parts = []
                        if problem_type:
                            input_parts.append(f"<problem>{problem_type}")

                        input_parts.append(f"<user>{combined_user_text}")

                        if emotion_type:
                            input_parts.append(f"<user_emotion>{emotion_type}")

                        input_parts.append("<therapist>")
                        input_text = " ".join(input_parts)

                        # Create evaluation sample
                        eval_sample = {
                            'input_text': input_text,
                            'reference_text': therapist_response,
                            'reference_emotion': emotion_type,  # Use conversation-level emotion
                            'user_input': combined_user_text,
                            'user_emotion': emotion_type,
                            'problem_type': problem_type,
                            'conversation_id': conv_idx,
                            'turn_id': turn_idx
                        }

                        self.evaluation_data.append(eval_sample)

                        # Show first few examples
                        if len(self.evaluation_data) <= 3:
                            print(f"\n  Sample {len(self.evaluation_data)}:")
                            print(f"    Input: {input_text[:100]}...")
                            print(f"    Reference: {therapist_response[:100]}...")

                        user_messages = []

        print(f"\nProcessed {len(self.evaluation_data)} evaluation samples from {len(self._data)} conversations")

    def get_evaluation_data(self):
        return self.evaluation_data

    def print_sample_formats(self, num_samples=3):
        """Print sample input/output formats for verification"""
        print(f"\n{'='*80}")
        print("ESCONV SAMPLE INPUT/OUTPUT FORMATS")
        print('='*80)

        for i, sample in enumerate(self.evaluation_data[:num_samples]):
            print(f"\nSample {i+1}:")
            print(f"  Problem Type: {sample['problem_type']}")
            print(f"  User Emotion: {sample['user_emotion']}")
            print(f"  Input Format: {sample['input_text'][:150]}...")
            print(f"  Reference Response: {sample['reference_text'][:100]}...")
            print(f"  User Input Only: {sample['user_input'][:100]}...")

# ====================== Model Format Detection ======================
def detect_model_format(model, tokenizer, device='cuda', test_inputs=None):
    """
    Test model output format (should be consistent since both SFT and RL use same format)

    Args:
        model: The model to test
        tokenizer: Model tokenizer
        device: Device to run test on
        test_inputs: List of test input strings, uses defaults if None

    Returns:
        str: 'standard' if using <therapist_emotion> format, 'unknown' otherwise
    """
    if test_inputs is None:
        test_inputs = [
            "<problem>anxiety <user>I'm worried about work <user_emotion>anxiety <therapist>",
            "<problem>depression <user>I feel very sad <user_emotion>sadness <therapist>",
            "<problem>relationship <user>My partner doesn't understand me <user_emotion>anger <therapist>"
        ]

    emotion_tag_count = 0
    total_tests = len(test_inputs)

    model.eval()
    with torch.no_grad():
        for test_input in test_inputs:
            # Tokenize and generate
            input_ids = tokenizer.encode(test_input, return_tensors='pt').to(device)
            outputs = model.generate(
                input_ids,
                max_new_tokens=128,
                do_sample=True,
                top_p=0.6,
                top_k=30,
                temperature=1.0,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

            # Decode response
            new_tokens = outputs[0][len(input_ids[0]):]
            response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

            # Check for emotion tag
            if '<therapist_emotion>' in response:
                emotion_tag_count += 1

    # Determine format
    if emotion_tag_count >= total_tests * 0.5:  # At least 50% have emotion tags
        return 'standard'
    else:
        return 'unknown'

# ====================== Model Output Extraction ======================
def extract_model_output(response):
    """
    Extract therapist text and emotion from model output.
    Both SFT and RL models use format: 'text <therapist_emotion> emotion<eos>'

    Returns:
        therapist_text (str): Text before <therapist_emotion>
        emotion (str): Emotion word after <therapist_emotion>
        has_emotion_tag (bool): Whether emotion tag was found
    """
    response = response.strip()

    # Remove <eos> if present
    if response.endswith('<eos>'):
        response = response[:-5].strip()

    # Find <therapist_emotion> tag
    emotion_pattern = r'<therapist_emotion>'
    emotion_match = re.search(emotion_pattern, response)

    if emotion_match:
        # Extract text before <therapist_emotion>
        therapist_text = response[:emotion_match.start()].strip()

        # Extract emotion part after <therapist_emotion>
        emotion_part = response[emotion_match.end():].strip()

        # Get the first word as emotion
        emotion_words = emotion_part.split()
        emotion = emotion_words[0].lower() if emotion_words else ""

        return therapist_text, emotion, True
    else:
        return response, "", False

# ====================== Evaluation Metrics ======================
class TherapyEvaluationMetrics:
    """Class to compute evaluation metrics"""

    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1

    def compute_bleu(self, reference, candidate):
        """Compute BLEU score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=self.smoothing)

    def compute_rouge(self, reference, candidate):
        """Compute ROUGE scores"""
        if not candidate or not reference:
            return {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}

        scores = self.rouge_scorer.score(reference, candidate)
        return {
            'rouge1': scores['rouge1'].fmeasure,
            'rouge2': scores['rouge2'].fmeasure,
            'rougeL': scores['rougeL'].fmeasure
        }

    def compute_meteor(self, reference, candidate):
        """Compute METEOR score"""
        if not candidate or not reference:
            return 0.0

        reference_tokens = reference.split()
        candidate_tokens = candidate.split()

        if len(candidate_tokens) == 0:
            return 0.0

        return meteor_score([reference_tokens], candidate_tokens)

# ====================== Model Evaluation ======================
def evaluate_model_esconv(model, tokenizer, evaluation_dataset, device='cuda', max_new_tokens=128,
                         top_p=0.6, top_k=30, temperature=1.0, do_sample=True):
    """
    Evaluate model on ESConv therapy dataset

    Args:
        model: Trained model (SFT or RL)
        tokenizer: Model tokenizer
        evaluation_dataset: ESConvEvaluationDataset instance
        device: Device to run evaluation on
        max_new_tokens: Maximum tokens to generate
        top_p: Top-p sampling parameter
        do_sample: Whether to use sampling

    Returns:
        Dictionary containing evaluation results
    """
    metrics_computer = TherapyEvaluationMetrics()
    evaluation_data = evaluation_dataset.get_evaluation_data()

    results = {
        'bleu_scores': [],
        'rouge1_scores': [],
        'rouge2_scores': [],
        'rougeL_scores': [],
        'meteor_scores': [],
        'emotion_accuracy': [],
        'has_emotion_tag': [],
        'sample_outputs': []
    }

    emotion_confusion = defaultdict(lambda: defaultdict(int))

    model.eval()

    # Detect model output format
    detected_format = detect_model_format(model, tokenizer, device)
    print(f"Output format check: {detected_format.upper()}")

    print(f"Evaluating model on {len(evaluation_data)} ESConv samples...")

    # Create progress bar with custom format
    progress_bar = tqdm(evaluation_data, desc="Evaluating ESConv",
                       bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]')

    for i, sample in enumerate(progress_bar):
        input_text = sample['input_text']
        reference_text = sample['reference_text']
        reference_emotion = sample['reference_emotion']

        # Tokenize input
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                do_sample=do_sample,
                top_p=top_p,
                top_k=top_k,
                temperature=temperature,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode response
        new_tokens = outputs[0][len(input_ids[0]):]
        response = tokenizer.decode(new_tokens, skip_special_tokens=False).strip()

        # Extract model output
        predicted_text, predicted_emotion, has_emotion = extract_model_output(response)

        # Compute text generation metrics
        bleu = metrics_computer.compute_bleu(reference_text, predicted_text)
        rouge = metrics_computer.compute_rouge(reference_text, predicted_text)
        meteor = metrics_computer.compute_meteor(reference_text, predicted_text)

        # Compute emotion accuracy
        emotion_correct = (predicted_emotion.lower() == reference_emotion.lower()) if has_emotion else False

        # Store results
        results['bleu_scores'].append(bleu)
        results['rouge1_scores'].append(rouge['rouge1'])
        results['rouge2_scores'].append(rouge['rouge2'])
        results['rougeL_scores'].append(rouge['rougeL'])
        results['meteor_scores'].append(meteor)
        results['emotion_accuracy'].append(emotion_correct)
        results['has_emotion_tag'].append(has_emotion)

        # Update confusion matrix
        if has_emotion:
            emotion_confusion[reference_emotion.lower()][predicted_emotion.lower()] += 1
        else:
            emotion_confusion[reference_emotion.lower()]['no_prediction'] += 1

        # Store sample outputs for inspection
        if i < 10:  # Store first 10 samples
            results['sample_outputs'].append({
                'input': input_text,
                'reference_text': reference_text,
                'predicted_text': predicted_text,
                'reference_emotion': reference_emotion,
                'predicted_emotion': predicted_emotion,
                'full_response': response,
                'bleu': bleu,
                'rouge1': rouge['rouge1'],
                'meteor': meteor
            })

        if (i + 1) % 20 == 0 or i < 5:
            sample_info = f"Sample {i+1}: BLEU={bleu:.3f}, Emotion={'✓' if emotion_correct else '✗'}"
            progress_bar.set_description(f"Evaluating ESConv - {sample_info}")

            if (i + 1) % 50 == 0 or i < 3:
                print(f"\n" + "="*80)
                print(f"ESCONV LIVE SAMPLE {i+1}/{len(evaluation_data)}")
                print("="*80)
                print(f"Input: {input_text[:120]}...")
                print(f"Reference: {reference_text[:100]}...")
                print(f"Generated: {predicted_text[:100]}...")
                print(f"Full Response: {response[:150]}...")
                print(f"Ref Emotion: {reference_emotion} | Pred Emotion: {predicted_emotion} | Match: {'✓' if emotion_correct else '✗'}")
                print(f"BLEU: {bleu:.4f} | ROUGE-1: {rouge['rouge1']:.4f} | METEOR: {meteor:.4f}")
                print("="*80)
        else:
            # Just update progress bar with running averages
            if i > 0:
                avg_bleu = np.mean(results['bleu_scores'])
                avg_emotion_acc = np.mean(results['emotion_accuracy'])
                progress_bar.set_description(f"Evaluating ESConv - Avg BLEU: {avg_bleu:.3f}, Emotion Acc: {avg_emotion_acc:.3f}")

    progress_bar.close()

    # Compute summary statistics
    results['summary'] = {
        'avg_bleu': np.mean(results['bleu_scores']),
        'avg_rouge1': np.mean(results['rouge1_scores']),
        'avg_rouge2': np.mean(results['rouge2_scores']),
        'avg_rougeL': np.mean(results['rougeL_scores']),
        'avg_meteor': np.mean(results['meteor_scores']),
        'emotion_accuracy': np.mean(results['emotion_accuracy']),
        'emotion_tag_coverage': np.mean(results['has_emotion_tag']),
        'total_samples': len(evaluation_data)
    }

    results['emotion_confusion_matrix'] = dict(emotion_confusion)
    results['detected_format'] = detected_format

    print(f"\n{'='*60}")
    print(f"ESCONV EVALUATION COMPLETED!")
    print(f"{'='*60}")
    print(f"Processed {len(evaluation_data)} samples")
    print(f"Average BLEU: {results['summary']['avg_bleu']:.4f}")
    print(f"Average ROUGE-1: {results['summary']['avg_rouge1']:.4f}")
    print(f"Emotion Accuracy: {results['summary']['emotion_accuracy']:.4f}")
    print(f"Emotion Tag Coverage: {results['summary']['emotion_tag_coverage']:.4f}")
    print(f"{'='*60}")

    return results

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    summary = results['summary']

    print("\n" + "="*60)
    print("ESCONV THERAPY MODEL EVALUATION RESULTS")
    print("="*60)
    print(f"Model Type: {results.get('model_type', 'Unknown').upper()}")
    print(f"Output Format: {results.get('detected_format', 'Unknown').upper()}")

    print(f"\nText Generation Metrics:")
    print(f"  BLEU Score:     {summary['avg_bleu']:.4f}")
    print(f"  ROUGE-1:        {summary['avg_rouge1']:.4f}")
    print(f"  ROUGE-2:        {summary['avg_rouge2']:.4f}")
    print(f"  ROUGE-L:        {summary['avg_rougeL']:.4f}")
    print(f"  METEOR:         {summary['avg_meteor']:.4f}")

    print(f"\nEmotion Prediction:")
    print(f"  Emotion Accuracy:    {summary['emotion_accuracy']:.4f}")
    print(f"  Emotion Tag Coverage: {summary['emotion_tag_coverage']:.4f}")

    print(f"\nDataset Info:")
    print(f"  Total Samples:  {summary['total_samples']}")

    # Print emotion confusion matrix
    if 'emotion_confusion_matrix' in results:
        print(f"\nEmotion Confusion Matrix:")
        confusion = results['emotion_confusion_matrix']
        for ref_emotion, pred_dict in confusion.items():
            print(f"  {ref_emotion}:")
            for pred_emotion, count in pred_dict.items():
                print(f"    -> {pred_emotion}: {count}")

    print(f"\nSample Outputs:")
    for i, sample in enumerate(results['sample_outputs'][:3], 1):
        print(f"\n  Sample {i}:")
        print(f"    Input: {sample['input'][:100]}...")
        print(f"    Reference: {sample['reference_text'][:80]}...")
        print(f"    Predicted: {sample['predicted_text'][:80]}...")
        print(f"    Ref Emotion: {sample['reference_emotion']}")
        print(f"    Pred Emotion: {sample['predicted_emotion']}")
        print(f"    BLEU: {sample['bleu']:.3f}, ROUGE-1: {sample['rouge1']:.3f}")

# ====================== Model Loading Functions ======================
def load_sft_model(checkpoint_path, tokenizer_path, device='cuda'):
    """Load SFT model from checkpoint file"""
    from transformers import GPT2LMHeadModel

    print(f"Loading SFT checkpoint: {checkpoint_path}")

    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.resize_token_embeddings(len(tokenizer))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)

    if 'epoch' in checkpoint:
        print(f"Checkpoint info: Epoch {checkpoint['epoch']}, Loss {checkpoint.get('valid_loss', 'N/A')}")

    return model, tokenizer

def load_rl_model(model_dir, tokenizer_path, device='cuda'):
    """Load RL model from directory"""
    from transformers import AutoModelForCausalLM

    print(f"Loading RL model from directory: {model_dir}")

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

    # Load model from directory
    model = AutoModelForCausalLM.from_pretrained(model_dir).to(device)

    return model, tokenizer

def load_model_auto(model_path, tokenizer_path, device='cuda'):
    """
    Automatically detect and load model (SFT checkpoint or RL directory)

    Args:
        model_path: Path to model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        device: Device to load model on

    Returns:
        tuple: (model, tokenizer, model_type)
    """
    import os

    if os.path.isfile(model_path):
        print("Detected SFT checkpoint file")
        model, tokenizer = load_sft_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'sft'

    elif os.path.isdir(model_path):
        print("Detected RL model directory")
        model, tokenizer = load_rl_model(model_path, tokenizer_path, device)
        return model, tokenizer, 'rl'

    else:
        raise ValueError(f"Model path {model_path} is neither a file nor a directory")

# ====================== Main Evaluation Function ======================
def run_esconv_evaluation(model_path, tokenizer_path, esconv_data_path, device='cuda'):
    """
    Run complete evaluation pipeline on ESConv data

    Args:
        model_path: Path to trained model (file for SFT, directory for RL)
        tokenizer_path: Path to tokenizer
        esconv_data_path: Path to ESConv JSON data
        device: Device to run evaluation on
    """

    # Load model and tokenizer (auto-detect type)
    print(f"Loading model from {model_path}...")
    model, tokenizer, model_type = load_model_auto(model_path, tokenizer_path, device)
    print(f"Loaded {model_type.upper()} model successfully")

    # Load ESConv evaluation dataset
    print(f"Loading ESConv evaluation data from {esconv_data_path}...")
    eval_dataset = ESConvEvaluationDataset(esconv_data_path, tokenizer_path)

    eval_dataset.print_sample_formats(3)

    results = evaluate_model_esconv(model, tokenizer, eval_dataset, device=device)
    results['model_type'] = model_type
    results['dataset_type'] = 'esconv'

    print_evaluation_results(results)

    return results

# ====================== Usage Example ======================
if __name__ == "__main__":

    MODEL_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt"
    TOKENIZER_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/train_processed_4thFIXED_tokenizer"
    ESCONV_DATA_PATH = "/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/PESConv.json"  # Update this path to your ESConv data
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

    results = run_esconv_evaluation(MODEL_PATH, TOKENIZER_PATH, ESCONV_DATA_PATH, DEVICE)

    import pickle
    with open('/content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/Evaluation2/ESConv_SFT_evaluation_results.pkl', 'wb') as f:
        pickle.dump(results, f)

    print("\nEvaluation completed! Results saved to ESConv_RL_evaluation_results.pkl")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading model from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt...
Detected SFT checkpoint file
Loading SFT checkpoint: /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/therapy_model_4thFIXED_epoch_7_loss_2.2373.ckpt


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Checkpoint info: Epoch 7, Loss 2.237253785133362
Loaded SFT model successfully
Loading ESConv evaluation data from /content/drive/MyDrive/RL-SFT-GPT2 MentalHealth/RL<token>8 6/PESConv.json...
Processing ESConv data...


Processing conversations:  28%|██▊       | 370/1300 [00:00<00:00, 3696.80it/s]

...


ESCONV LIVE SAMPLE 14500/14642
Input: <problem>breakup with partner <user>Yeah she was a little violent. I was honestly a little scared for my life for a seco...
Reference: Have you talked since then?...
Generated: And how did you react when she did threaten to kill you?...
Full Response: And how did you react when she did threaten to kill you? <therapist_emotion> neutral<eos>...
Ref Emotion: sadness | Pred Emotion: neutral | Match: ✗
BLEU: 0.0170 | ROUGE-1: 0.1176 | METEOR: 0.0877


Evaluating ESConv - Sample 14600: BLEU=0.001, Emotion=✗: 100%|█████████▉| 14600/14642 [31:24<00:04,  9.65it/s]


ESCONV LIVE SAMPLE 14600/14642
Input: <problem>job crisis <user>Yes It is frustrating. When I think about the financial issues I feel depressed <user_emotion>...
Reference: I compleyely understand I have also lost my career due to covid. Dont give up, staying positive and ...
Generated: I can imagine that you feel...
Full Response: I can imagine that you feel  <therapist_emotion> neutral<eos>...
Ref Emotion: depression | Pred Emotion: neutral | Match: ✗
BLEU: 0.0012 | ROUGE-1: 0.1176 | METEOR: 0.0991


Evaluating ESConv - Avg BLEU: 0.005, Emotion Acc: 0.008: 100%|██████████| 14642/14642 [31:29<00:00,  7.75it/s]



ESCONV EVALUATION COMPLETED!
Processed 14642 samples
Average BLEU: 0.0050
Average ROUGE-1: 0.1046
Emotion Accuracy: 0.0080
Emotion Tag Coverage: 0.9979

ESCONV THERAPY MODEL EVALUATION RESULTS
Model Type: SFT
Output Format: STANDARD

Text Generation Metrics:
  BLEU Score:     0.0050
  ROUGE-1:        0.1046
  ROUGE-2:        0.0089
  ROUGE-L:        0.0925
  METEOR:         0.0354

Emotion Prediction:
  Emotion Accuracy:    0.0080
  Emotion Tag Coverage: 0.9979

Dataset Info:
  Total Samples:  14642

Emotion Confusion Matrix:
  anxiety:
    -> neutral: 3359
    -> but: 24
    -> <therapist_emotion>: 70
    -> i: 66
    -> tell: 1
    -> i'm: 49
    -> what: 23
    -> i'm...: 2
    -> who: 11
    -> well,: 10
    -> that: 4
    -> i'll: 2
    -> anger?: 11
    -> and: 13
    -> unwell.: 1
    -> okay,: 2
    -> get: 2
    -> you: 29
    -> cause: 10
    -> unrelated?: 1
    -> you're: 18
    -> you've: 6
    -> ?: 7
    -> anger.: 11
    -> so: 14
    -> but...: 2
    -> .: 6
    -> ap